This notebook will ideally be run only one time to download and set up the pangenome and annotation data needed to construct and analyze the E. coli non-coding alleleome.

# Import Packages

In [145]:
# built-in packages
import gzip
import os
from pathlib import Path
import pickle
import subprocess
import sys

# third-party packages
from matplotlib.cbook import boxplot_stats
import numpy as np
import pandas as pd

# import local pangenomix
sys.path.append('pangenomix/')
from pangenome import build_cds_pangenome, load_sequences_from_fasta

# import local mutil
sys.path.append('mutil/')

from mutil.alemutdf import get_all_sample_mut_df, get_multi_exp_max_freq_mut_df
from mutil.genome import get_K12_pos_from_BOP27
from mutil.mut import get_clean_mut_gene_list, is_coding_mut

# import local bitome2
sys.path.append('bitome2/')

from bitome.core import Bitome

# import local util functions
from util import load_gff, write_fasta_lines

# Set Paths

This will set paths on your local machine that this workflow needs to be aware of - see comments in this cell to understand which folders you need to create

In [108]:
# overall general data directory within this repository
DATA_DIR = Path('data/')

# directory for all large files; this data will be LARGE (~20-25 GB) so choose/generate 
# a directory that is not in this repository that has a lot of storage
LARGE_DATA_DIR = Path('/Users/cam/Projects/noncoding_alleleome_large_data/')
PANGENOME_DIR = Path(LARGE_DATA_DIR, 'pangenome')
# this will store files used for phylogrouping the strains
PHYLOGROUPING_DIR = Path(LARGE_DATA_DIR, 'phylogrouping')
# this will store files related to the coding pangenome
CODING_PANGENOME_DIR = Path(LARGE_DATA_DIR, 'cds_pangenome')
# this directory will store blast results organized by reference feature
BLAST_BY_UPSTREAM_REGION_DIR = Path(LARGE_DATA_DIR, 'blast_by_upstream_region')
# this will store ALE mutation files
ALE_DIR = Path(LARGE_DATA_DIR, 'ale')

# this contains tables used to generate the bitome object
BITOME_DATA_PATH = Path(DATA_DIR, 'bitome')

# this path will contain the input FASTA files from the reference organism for blasting
BLAST_INPUT_DIR = Path(DATA_DIR, 'blast_input')

# this path will be used to write temporary files for alignment with MUSCLE
# allowing us to collapse duplicate alleles
ALIGN_TEMP_DIR = Path(DATA_DIR, 'align_tmp')

# Download Genomes from BV-BRC

Load pre-downloaded metadata table of complete E. coli genomes of good quality. This table should be downloaded from BV-BRC (formerly known as PATRIC) with the following steps:

- Search "Escherichia coli" in the upper search bar
- From "Taxon View" main dashboard page for E. coli, click the "Genomes" tab
- Click "Filters" in the top right
- Under "Genome Status" click "Complete"
- Under "Genome Quality" click "Good"
- Click "Download" in the top left corner; this will download it to the "data" directory of this repository

## Load Genome Metadata

Load in the file with genome IDs and metadata that was downloaded from BV-BRC

In [4]:
ecoli_genome_md_df = pd.read_excel(Path(DATA_DIR, 'bv-brc_e_coli_complete_genome_metadata.xlsx'), index_col=None, dtype='object')
ecoli_genome_md_df = ecoli_genome_md_df.set_index('Genome ID')
ecoli_genome_md_df.head()

,Genome Name,Other Names,NCBI Taxon ID,Taxon Lineage IDs,Taxon Lineage Names,Superkingdom,Kingdom,Phylum,Class,Order,...,Host Age,Host Health,Host Group,Lab Host,Passage,Other Clinical,Additional Metadata,Comments,Date Inserted,Date Modified
Genome ID,,,,,,,,,,,,,,,,,,,,,
1033813.3,Escherichia coli NA114,NaN,1033813,131567;2;1224;1236;91347;543;561;562;1033813,cellular organisms;Bacteria;Proteobacteria;Gam...,Bacteria,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bacterial culture archived with Pathogen Biolo...,2014-12-08T22:11:31.206Z,2015-03-16T03:17:09.594Z
1038927.31,Escherichia coli O104:H4 strain FDAARGOS_348,NaN,1038927,131567;2;1224;1236;91347;543;561;562;1038927,cellular organisms;Bacteria;Proteobacteria;Gam...,Bacteria,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,...,NaN,Bloody diarrhea complicated hemolytic uremic s...,Human,NaN,NaN,NaN,collected_by:USAMRIID < CDC-Atlanta,"In May 2014, the FDA and collaborators establi...",2017-07-23T01:19:19.428Z,2017-07-23T01:19:19.428Z
1038927.40,Escherichia coli O104:H4 strain FDAARGOS_349,NaN,1038927,131567;2;1224;1236;91347;543;561;562;1038927,cellular organisms;Bacteria;Proteobacteria;Gam...,Bacteria,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,...,37Y,Hemolytic uremic syndrome,Human,NaN,NaN,NaN,collected_by:USAMRIID < Bundeswehr Inst. Micro...,"In May 2014, the FDA and collaborators establi...",2018-05-26T05:50:28.248Z,2018-05-26T05:50:28.248Z
1038927.41,Escherichia coli O104:H4 strain LB226692,NaN,1038927,131567;2;1224;1236;91347;543;561;562;1038927,cellular organisms;Bacteria;Proteobacteria;Gam...,Bacteria,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,...,NaN,NaN,Human,NaN,NaN,NaN,collected_by:German National Consulting Labora...,Study of phage-borne NanS-p esterases.,2018-05-30T17:27:28.566Z,2018-05-30T17:27:28.566Z
1038927.45,Escherichia coli O104:H4 strain FWSEC0009,NaN,1038927,131567;2;1224;1236;91347;543;561;562;1038927,cellular organisms;Bacteria;Proteobacteria;Gam...,Bacteria,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,...,NaN,NaN,Human,NaN,NaN,NaN,collected_by:Laboratory for Foodborne Zoonoses...,The food and water safety pilot project is a c...,2019-07-14T19:20:31.213Z,2019-07-14T19:20:31.213Z


## Create Subdirectories in Large Data Directory

This will set up a subdirectory for each genome ID in the large data pangenome folder

In [120]:
for genome_id in ecoli_genome_md_df.index:
    Path(PANGENOME_DIR, genome_id).mkdir(exist_ok=True)

## Download Genomes and Annotation

This short bash script will loop through each of the above-created subdirectories to download the relevant genomes

This is meant to be run directly from the command line - so copy this code, use a Terminal to navigate to `PANGENOME_DIR`, and execute this code there

This step may take a while depending on your Internet speed

Triple check that you have ~20-25 GB of storage available in `PANGENOME_DIR`

In [ ]:
for i in *;
do
cd $i
wget "ftp://ftp.bvbrc.org/genomes/$i/$i.fna"
wget "ftp://ftp.bvbrc.org/genomes/$i/$i.PATRIC.faa"
wget "ftp://ftp.bvbrc.org/genomes/$i/$i.PATRIC.gff"
cd ..
done

# QC Genomes

Now we want to check some characteristics of the genomes/annotations we downloaded to ensure we only analyze high-quality genomes

## Check for Ambiguous Bases

We only want genomes that are composed entirely of ACGT bases - no N etc.

In [121]:
seq_file_dicts = {
    genome_dir: {
        'faa': Path(PANGENOME_DIR, genome_dir, f'{genome_dir}.PATRIC.faa'),
        'fna': Path(PANGENOME_DIR, genome_dir, f'{genome_dir}.fna'),
        'gff': Path(PANGENOME_DIR, genome_dir, f'{genome_dir}.PATRIC.gff')
    }
    for genome_dir in os.listdir(PANGENOME_DIR) if genome_dir != '.DS_Store'
}
    

# make sure files actually exist
seq_file_dicts_exist = filter(lambda tup: all([path.exists() for path in tup[1].values()]), seq_file_dicts.items())

# make sure the FAA is not empty (implies GFF also empty)
def non_empty_faa(faa):
    with open(faa, 'r') as f:
        return len(f.readlines()) > 0
seq_file_dicts_with_cds = filter(lambda tup: non_empty_faa(tup[1]['faa']), seq_file_dicts_exist)

# we also have to make sure we only have ACGT sequences in the FNA
def only_acgt_fna(fna):

    seq_dict = load_sequences_from_fasta(fna)
    # if no seqs, skip as well
    if len(seq_dict) == 0:
        return False
    for seq_name, seq in seq_dict.items():
        if set(seq.lower()) != {'a', 'c', 'g', 't'}:
            return False
    return True

seq_file_dicts_with_acgt = dict(list(filter(lambda tup: only_acgt_fna(tup[1]['fna']), seq_file_dicts_with_cds)))

# write the resultant valid genome IDs to a file for later use
valid_file_table = pd.DataFrame(seq_file_dicts_with_acgt).T

## QC L50/N50 Stats

Here we'll select only genomes with a single contig (complete assemblies) and the appropriate genome length (>4M)

Note that this still allows genomes with plasmids

In [1311]:
val_idx = valid_file_table['fna'].apply(lambda s: s.split('/')[-2])
patric_val_md_df = ecoli_genome_md_df.loc[val_idx]
valid_file_table = valid_file_table[((patric_val_md_df['Contig L50'] == 1) & (patric_val_md_df['Contig N50'] > 4000000)).values]

# ClermonTyping for Phylogroups

Determine phylogroups for QCed strains using the ClermonTyping tool

## Write Genome to FNA Lookup

In [56]:
# Location of genomes
with open(Path(PHYLOGROUPING_DIR, 'prokka_fna_dict.pickle'), 'wb') as f:
    genome_fna_dict = {}
    for _, vf_row in valid_file_table.iterrows():
        fna = vf_row['fna']
        gid = fna.split('/')[-2]
        genome_fna_dict[gid] = fna
    pickle.dump(genome_fna_dict, f)

## Run ClermonTyping Phylogrouping Scripts (must be run in `PHYLOGROUPING_DIR`)

These scripts are adapted from the coding pangenome workflow and must be run in the same worklow as the eventual outputs

- Move the 2 files from the `phylogrouping_scripts` directory in this repository into the `PHYLOGROUPING_DIR` directory you created earlier
- Run `1__annotate_strains.py` from the command line
- Run through all cells of `2__parse_annotation.ipynb`, again still within `PHYLOGROUPING_DIR` (you will need to open up a separate Jupyter Lab session from that directory for this)
- The result of these steps will be an `ecoli_phylogroups.csv` data table in the `PHYLOGROUPING_DIR` that relates genome IDs to phylogroup assignments

## Load Processed Phylogroup Info

In [7]:
phylogroup_df = pd.read_csv(Path(PHYLOGROUPING_DIR, 'ecoli_phylogroups.csv'), index_col=0, dtype='object')
phylogroup_df.head()

,genome_id,pcr_genes,quadruplex,CE_alleles,phylogroup
0,562.46071,"['ybgD', 'trpA', 'trpBA', 'yjaA', 'arpA']","['+', '-', '+', '-']",[],A
1,562.70513,"['ybgD', 'trpA', 'trpBA', 'chuA', 'TspE4.C2', ...","['+', '+', '-', '+']",['trpAgpC'],D
2,562.67796,"['ybgD', 'trpA', 'trpBA', 'TspE4.C2', 'arpA', ...","['+', '-', '-', '+']",['trpAgpC'],B1
3,83334.637,"['trpA', 'trpBA', 'chuA', 'arpA', 'ArpAgpE', '...","['+', '+', '-', '-']","['ArpAgpE', 'trpAgpC']",E
4,562.102351,"['ybgD', 'trpA', 'trpBA', 'chuA', 'arpA', 'trp...","['+', '+', '-', '-']",['trpAgpC'],D


## Remove Unknown/Non EC Genomes

In [ ]:
bad_genome_ids = phylogroup_df[phylogroup_df['phylogroup'].isin(['Non Escherichia', 'Unknown'])]['genome_id']

valid_file_table = valid_file_table[
    ~valid_file_table['fna'].apply(lambda f: f.split('/')[-2]).isin(bad_genome_ids.values)
]
valid_genome_md = ecoli_genome_md_df.loc[valid_file_table['fna'].apply(lambda f: f.split('/')[-2])]
phylo_lookup = dict(zip(phylogroup_df['genome_id'], phylogroup_df['phylogroup']))
valid_genome_md = valid_genome_md.assign(
    phylogroup=pd.Series(valid_genome_md.index).apply(lambda gid: phylo_lookup[gid]).values
)
valid_genome_md = valid_genome_md.assign(
    genome_id=valid_genome_md.index
)
valid_genome_md = valid_genome_md.reset_index()
valid_genome_md = valid_genome_md.drop(columns='Genome ID')

# join together
valid_file_table = valid_file_table.assign(
    genome_id=valid_file_table['fna'].apply(lambda f: f.split('/')[-2])
)
valid_genome_md = valid_genome_md.merge(valid_file_table, how='inner', on='genome_id')

## Write Valid Genome Info to Files

In [109]:
valid_file_table.to_csv(Path(DATA_DIR, 'valid_files.csv'))
valid_genome_md.to_csv(Path(DATA_DIR, 'valid_genome_md.csv'))

# Generate CDS Pangenome

The coding sequence pangenome is necessary in order to define gene clusters across strains. Gene clusters containing genes from the reference strain K-12 MG1655 will be used to search for K-12 MG1655 non-coding regions upstream of all genes in the pangenome cluster

In [115]:
build_cds_pangenome(list(valid_genome_md['faa']), str(CODING_PANGENOME_DIR),
                   cdhit_args={'-n': 5, '-c': 0.8}, name=f'BV-BRC_{valid_genome_md.shape[0]}')

Identifying non-redundant CDS sequences...
Headers without sequences: 13784
Running: ['cd-hit', '-i', '../data/pangenome_output/BV-BRC_2350_nr.faa', '-o', '../data/pangenome_output/BV-BRC_2350_nr.faa.cdhit', '-d', '0', '-n', '5', '-c', '0.8']
Program: CD-HIT, V4.8.1 (+OpenMP), Mar 21 2023, 16:31:39
Command: cd-hit -i ../data/pangenome_output/BV-BRC_2350_nr.faa
         -o ../data/pangenome_output/BV-BRC_2350_nr.faa.cdhit
         -d 0 -n 5 -c 0.8

Started: Sat Jul  8 11:05:07 2023
                            Output
----------------------------------------------------------------
total seq: 918781
longest and shortest : 7227 and 11
Total letters: 315522850
Sequences have been sorted

Approximated minimal memory consumption:
Sequence        : 449M
Buffer          : 1 X 23M = 23M
Table           : 1 X 80M = 80M
Miscellaneous   : 11M
Total           : 563M

Table limit with the given memory limit:
Max number of representatives: 414626
Max number of word counting entries: 29553902

comparin

(<sparse_utils.LightSparseDataFrame at 0x921243e20>,
 <sparse_utils.LightSparseDataFrame at 0x921243c40>)

## Identify Gene Clusters with K-12 Strain Involved

If you're adapting this workflow for a different organism, you'll need to modify this cell to identify gene clusters with alleles in your reference organism of interest (i.e. the reference organism that has the best existing non-coding feature annotations, TSS etc, for knowledge mapping)

In [7]:
# we want to get all clusters mapped to all genes
cluster_to_genes = {}

#with open(Path(CODING_PANGENOME_DIR, f'BV-BRC_{valid_genome_md.shape[0]}_allele_names.tsv'), 'r') as f:
with open('/Users/cam/Projects/Strainware-s/workflows_CAD/data/pangenome_output/BV-BRC_2350_allele_names.tsv', 'r') as f:
    
    current_line = f.readline()
    count = 0
    while current_line:
        allele_id, genes_raw = current_line.strip().split('\t', maxsplit=1)
        cluster_id = allele_id.split('A')[0]
        genes = genes_raw.split('\t')
        if cluster_id in cluster_to_genes:
            cluster_to_genes[cluster_id].extend(genes)
        else:
            cluster_to_genes[cluster_id] = genes
        current_line = f.readline()
        
# then only take clusters that have an MG1655 gene
k12_bvbrc_id = '511145.12'
clusters_with_mg = {
    k: list(set(v))
    for k, v in cluster_to_genes.items()
    if k12_bvbrc_id in ''.join(list(set(v)))
}

# now we want to associate each MG1655 b-number with all the pangenome genes it clustered with
bnum_to_genes = {}

for cluster_id, cluster_genes in clusters_with_mg.items():
    # isolate the MG genes (must be at least one)
    mg_genes = [g for g in cluster_genes if k12_bvbrc_id in g and '|b' in g]
    for mg_gene in mg_genes:
        bnum = mg_gene.split('|')[-1]
        if bnum in bnum_to_genes:
            bnum_to_genes[bnum].extend(cluster_genes)
        else:
            bnum_to_genes[bnum] = cluster_genes
            
# drop the locus tag from these genes to get just the ID as it will appear in GFFs
def drop_locus_tag(gene):
    prefix, genome_peg_locus = gene.split('|', maxsplit=1)
    if '|' in genome_peg_locus:
        genome_peg = genome_peg_locus.split('|')[0]
    else:
        genome_peg = genome_peg_locus
    return f"{prefix}|{genome_peg}"
bnum_to_genes = {bnum: [drop_locus_tag(g) for g in genes] for bnum, genes in bnum_to_genes.items()}

# we also want genes to bnum
# can't assume uniquely one b-num per pan-gene
pan_gene_to_bnums = {}
for bnum, pan_genes in bnum_to_genes.items():
    for pan_gene in pan_genes:
        if pan_gene in pan_gene_to_bnums:
            pan_gene_to_bnums[pan_gene].append(bnum)
        else:
            pan_gene_to_bnums[pan_gene] = [bnum]

# Create Genome Feature Tables

We want to do a one-time parse of the GFF files into a more usable state

In [ ]:
for i, gff_file in enumerate(valid_genome_md['gff']):
    print(f'{i+1}/{valid_genome_md.shape[0]}', end='\r')
    gff_df = load_gff(gff_file)
    gff_df = gff_df.drop(columns=['score', 'phase'])

    def attr_to_dict(attrs):
        if isinstance(attrs, str):
            return {attr.split('=')[0]: attr.split('=')[1] for attr in attrs.split(';') if '=' in attr}
        else:
            return {}

    attr_df = pd.DataFrame(list(gff_df['attributes'].apply(attr_to_dict).values))
    full_df = pd.concat([gff_df, attr_df], axis=1)
    full_df = full_df.assign(
        feature_id=full_df['ID'].apply(
            lambda full_id: full_id.split('|')[1].replace(f"{'562.30434'}.", '')
            if isinstance(full_id, str) else np.nan
        )
    )
    
    genome_dir = Path(gff_file).parent
    full_df.to_csv(Path(genome_dir, 'feature_table.csv'))

# Process ALE Mutations

These must be downloaded from [ALEdb](https://aledb.org/)

- Click "See All Available Projects" on the main page
- Click "Experiments" on the left sidebar
- Click "Select" and "Select All"
- Click "Export" and "Mutations" -> when prompted, save to a subfolder called `mutations_raw` in the `ALE_DIR` you defined earlier
    - This may take a few minutes to complete

    
The following steps will perform curation of the downloaded mutations

## Consolidate and Process ALE Mutation Tables

In [10]:
muts_df = get_all_sample_mut_df(str(Path(ALE_DIR, 'mutations_raw')))

# only want MG1655 mutations
muts_df = muts_df[muts_df['Reference Seq'] == 'NC_000913']

def get_exp_sample_name(exp_name, ale, flask, isolate, tech_rep):
    sample_name = exp_name + " " + str(int(ale)) + " " + str(int(flask)) + " " + str(int(isolate)) + " " + str(int(tech_rep))
    return sample_name

muts_df["sample"] = muts_df.apply(lambda r: get_exp_sample_name(r.exp, r.ale, r.flask, r.isolate, r.tech_rep), axis=1)

# Different experiments have different strings for position (some with commas, some without), therefore going ahead and changing them all to integers
muts_df.Position = muts_df.Position.apply(lambda x: int(str(x).replace(",","")))
muts_df.Position = muts_df.Position.astype(int)

# ensure all mutations are using NC_000913 positions.
genome_pos_transform_exp_l = [
    "ERS",
]
muts_df.Position = muts_df.apply(lambda row: get_K12_pos_from_BOP27(row.Position) if row.exp in genome_pos_transform_exp_l else row.Position, axis=1)

# removing unnecessary column
muts_df = muts_df.drop(columns=["Mut ID"])

muts_df["exp ALE"] = muts_df.apply(lambda m: m["exp"] + " " + str(m["ale"]),axis = 1)

Issue with file: ../data/ale/mutations_raw/SvNS_tpiKO_mut.csv
invalid literal for int() with base 10: '1.1'
None
Issue with file: ../data/ale/mutations_raw/SvNS_pgiKO_mut.csv
invalid literal for int() with base 10: '1.1'
None


Traceback (most recent call last):
  File "/Users/cam/Projects/Strainware-s/workflows_CAD/promoter_survey/../../mutil/alemutdf.py", line 93, in get_mut_dataframe
    df['tech_rep'] = int(col.split(' ')[-1][1:])
ValueError: invalid literal for int() with base 10: '1.1'
Traceback (most recent call last):
  File "/Users/cam/Projects/Strainware-s/workflows_CAD/promoter_survey/../../mutil/alemutdf.py", line 93, in get_mut_dataframe
    df['tech_rep'] = int(col.split(' ')[-1][1:])
ValueError: invalid literal for int() with base 10: '1.1'


Issue with file: ../data/ale/mutations_raw/SvNS_pgiVCH_mut.csv
invalid literal for int() with base 10: '1.1'
None


Traceback (most recent call last):
  File "/Users/cam/Projects/Strainware-s/workflows_CAD/promoter_survey/../../mutil/alemutdf.py", line 93, in get_mut_dataframe
    df['tech_rep'] = int(col.split(' ')[-1][1:])
ValueError: invalid literal for int() with base 10: '1.1'


Issue with file: ../data/ale/mutations_raw/SvNS_tpiPAE_mut.csv
invalid literal for int() with base 10: '1.1'
None
Issue with file: ../data/ale/mutations_raw/SvNS_pgiBME_mut.csv
invalid literal for int() with base 10: '1.1'
None


...

Traceback (most recent call last):
  File "/Users/cam/Projects/Strainware-s/workflows_CAD/promoter_survey/../../mutil/alemutdf.py", line 93, in get_mut_dataframe
    df['tech_rep'] = int(col.split(' ')[-1][1:])
ValueError: invalid literal for int() with base 10: '1.1'


Issue with file: ../data/ale/mutations_raw/SvNS_tpiHSA_mut.csv
invalid literal for int() with base 10: '1.1'
None


Traceback (most recent call last):
  File "/Users/cam/Projects/Strainware-s/workflows_CAD/promoter_survey/../../mutil/alemutdf.py", line 93, in get_mut_dataframe
    df['tech_rep'] = int(col.split(' ')[-1][1:])
ValueError: invalid literal for int() with base 10: '1.1'


Issue with file: ../data/ale/mutations_raw/SvNS_pgiHSA_mut.csv
invalid literal for int() with base 10: '1.1'
None


Traceback (most recent call last):
  File "/Users/cam/Projects/Strainware-s/workflows_CAD/promoter_survey/../../mutil/alemutdf.py", line 93, in get_mut_dataframe
    df['tech_rep'] = int(col.split(' ')[-1][1:])
ValueError: invalid literal for int() with base 10: '1.1'


## Remove Known Problematic Samples

In [11]:
muts_df = muts_df[~(
# For whatever reason, the gene annotations can change between version of breseq or runs
#     ((muts_df.Gene == "gatC")
#      | (muts_df.Gene == "gatC, gatC"))
    ((muts_df.Position == 2173361)
       | (muts_df.Position == 2173363)
       | (muts_df.Position == 2173364))
    & ((muts_df["Sequence Change"] == "Δ2 bp")
       | (muts_df["Sequence Change"] == "Δ1 bp"))
)]

muts_df = muts_df[~(
#     (muts_df.Gene == "[crl]")  # For whatever reason, the gene annotations can change between version of breseq or runs
    (muts_df.Position == 257908)
    & (muts_df["Sequence Change"] == "Δ776 bp"))]

muts_df = muts_df[~(
#     (muts_df.Gene == "insB1, insA")  # For whatever reason, the gene annotations can change between version of breseq or runs
    (muts_df.Position == 1978503)
    & (muts_df["Sequence Change"] == "Δ776 bp"))]

muts_df = muts_df[~(
#     (muts_df.Gene == "ychE, oppA")  # For whatever reason, the gene annotations can change between version of breseq or runs
    (muts_df.Position == 1299499)
    & (muts_df["Sequence Change"] == "Δ1,199 bp"))]

muts_df = muts_df[~(
# For whatever reason, the gene annotations can change between version of breseq or runs
#     ((muts_df.Gene == "glpR")
#      | (muts_df.Gene == "glpR, glpR")
#      | (muts_df.Gene == "glpR/glpR"))
    (muts_df.Position == 3560455)
    & (muts_df["Sequence Change"] == "+G"))]

muts_df = muts_df[~(
#     (muts_df.Gene == "gltP, yjcO")  # For whatever reason, the gene annotations can change between version of breseq or runs
    (muts_df.Position == 4296381)
    & (muts_df["Sequence Change"] == "+GC"))]

muts_df = muts_df[~(
#     (muts_df.Gene == "gltP, yjcO")  # For whatever reason, the gene annotations can change between version of breseq or runs
    (muts_df.Position == 4296380)
    & (muts_df["Sequence Change"] == "+CG"))]

muts_df = muts_df[~(
#     (muts_df.Gene == "insB1,insA")  # For whatever reason, the gene annotations can change between version of breseq or runs
    (muts_df.Position == 1978503)
    & (muts_df["Sequence Change"] == "Δ776 bp"))]

muts_df = muts_df[~(
#     ((muts_df.Gene == "insA/uspC") | (muts_df.Gene == "insA, uspC"))  # For whatever reason, the gene annotations can change between version of breseq or runs
    (muts_df.Position == 1979486)
    & (muts_df["Sequence Change"] == "IS5 (+) +4 bp"))]


muts_df = muts_df[~(
#     (muts_df.Gene == "corA")  # For whatever reason, the gene annotations can change between version of breseq or runs
    (muts_df.exp != "GLU")  # This is the ALE that originated the corA mutation that is found in all CCK samples, therefore should track
    & (muts_df.Position == 4001645)
    & (muts_df["Sequence Change"] == "Δ5 bp"))]


muts_df = muts_df[~(
#     (muts_df.Gene == "wcaA")  # For whatever reason, the gene annotations can change between version of breseq or runs
    (muts_df.exp != "GLU")  # This is the ALE that originated the corA mutation that is found in all CCK samples, therefore should track
    & (muts_df.Position == 2132787)
    & (muts_df["Details"] == "I204S (ATC→AGC)"))]


p = {4293212, 4293403, 4293430, 4296060, 4296154, 4296189,
 4296190, 4296191, 4296267, 4296268, 4296286, 4296380, 4296382}
ch = {'+C', '+G', 'A→C', 'A→G', 'A→T', 'C→A', 'C→T', 'T→C'}
muts_df = muts_df[~(
#     (muts_df.Gene == "gltP, yjcO")
    (muts_df["Position"].isin(p))
    & (muts_df["Sequence Change"].isin(ch))
       )]

reduced_ecoli_ale = 'application_of_ALE_to_re-optimize_growth_performance_of_genome-reduced_strain_MS56-M9-MG1655gbk'
MG1655_pBGT_untreated = "MG1655-pBGT_untreated"
MG1655_untreated = "MG1655_untreated"
MG1655_blaTEM1_untreated = "MG1655-blaTEM1_untreated"
muts_to_remove = [
    {"experiment":MG1655_blaTEM1_untreated,"position":1397381,"sequence change":"Δ13,756 bp"},
    {"experiment":MG1655_untreated,"position":1397381,"sequence change":"Δ13,756 bp"},
    {"experiment":MG1655_pBGT_untreated,"position":1397381,"sequence change":"Δ13,756 bp"},
    {"experiment":reduced_ecoli_ale,"position":389898,"sequence change":"Δ9,908 bp"},
    {"experiment":reduced_ecoli_ale,"position":1388889,"sequence change":"Δ9,734 bp"},
    {"experiment":reduced_ecoli_ale,"position":2466545,"sequence change":"Δ9,634 bp"},
    {"experiment":reduced_ecoli_ale,"position":729365,"sequence change":"Δ9,598 bp"},
    {"experiment":reduced_ecoli_ale,"position":3815859,"sequence change":"Δ82 bp"},
    {"experiment":reduced_ecoli_ale,"position":1400325,"sequence change":"Δ81,931 bp"},
    {"experiment":reduced_ecoli_ale,"position":380112,"sequence change":"Δ8,537 bp"},
    {"experiment":reduced_ecoli_ale,"position":2509631,"sequence change":"Δ8,319 bp"},
    {"experiment":reduced_ecoli_ale,"position":3761993,"sequence change":"Δ8,250 bp"},
    {"experiment":reduced_ecoli_ale,"position":257908,"sequence change":"Δ776 bp"},
    {"experiment":reduced_ecoli_ale,"position":1978503,"sequence change":"Δ776 bp"},
    {"experiment":reduced_ecoli_ale,"position":1040502,"sequence change":"Δ73,763 bp"},
    {"experiment":reduced_ecoli_ale,"position":3090347,"sequence change":"Δ708 bp"},
    {"experiment":reduced_ecoli_ale,"position":675552,"sequence change":"Δ7,819 bp"},
    {"experiment":reduced_ecoli_ale,"position":4400289,"sequence change":"Δ7 bp"},
    {"experiment":reduced_ecoli_ale,"position":819776,"sequence change":"Δ62,977 bp"},
    {"experiment":reduced_ecoli_ale,"position":263514,"sequence change":"Δ61,897 bp"},
    {"experiment":reduced_ecoli_ale,"position":3184782,"sequence change":"Δ6,916 bp"},
    {"experiment":reduced_ecoli_ale,"position":997868,"sequence change":"Δ6,818 bp"},
    {"experiment":reduced_ecoli_ale,"position":2558699,"sequence change":"Δ6,790 bp"},
    {"experiment":reduced_ecoli_ale,"position":3618992,"sequence change":"Δ6,688 bp"},
    {"experiment":reduced_ecoli_ale,"position":4496676,"sequence change":"Δ53,036 bp"},
    {"experiment":reduced_ecoli_ale,"position":167484,"sequence change":"Δ5,964 bp"},
    {"experiment":reduced_ecoli_ale,"position":603467,"sequence change":"Δ5,885 bp"},
    {"experiment":reduced_ecoli_ale,"position":3362162,"sequence change":"Δ5,481 bp"},
    {"experiment":reduced_ecoli_ale,"position":765208,"sequence change":"Δ5,405 bp"},
    {"experiment":reduced_ecoli_ale,"position":15389,"sequence change":"Δ5,175 bp"},
    {"experiment":reduced_ecoli_ale,"position":1509562,"sequence change":"Δ46,201 bp"},
    {"experiment":reduced_ecoli_ale,"position":332366,"sequence change":"Δ44,951 bp"},
    {"experiment":reduced_ecoli_ale,"position":4555491,"sequence change":"Δ41,523 bp"},
    {"experiment":reduced_ecoli_ale,"position":508873,"sequence change":"Δ40,744 bp"},
    {"experiment":reduced_ecoli_ale,"position":1223090,"sequence change":"Δ4 bp"},
    {"experiment":reduced_ecoli_ale,"position":2101397,"sequence change":"Δ36,321 bp"},
    {"experiment":reduced_ecoli_ale,"position":2756160,"sequence change":"Δ35,091 bp"},
    {"experiment":reduced_ecoli_ale,"position":2204350,"sequence change":"Δ32,368 bp"},
    {"experiment":reduced_ecoli_ale,"position":2994992,"sequence change":"Δ3,877 bp"},
    {"experiment":reduced_ecoli_ale,"position":2286401,"sequence change":"Δ3,780 bp"},
    {"experiment":reduced_ecoli_ale,"position":1997112,"sequence change":"Δ26,567 bp"},
    {"experiment":reduced_ecoli_ale,"position":1197137,"sequence change":"Δ25,940 bp"},
    {"experiment":reduced_ecoli_ale,"position":4299628,"sequence change":"Δ25,538 bp"},
    {"experiment":reduced_ecoli_ale,"position":1627518,"sequence change":"Δ25,244 bp"},
    {"experiment":reduced_ecoli_ale,"position":565055,"sequence change":"Δ21,054 bp"},
    {"experiment":reduced_ecoli_ale,"position":2846301,"sequence change":"Δ20,968 bp"},
    {"experiment":reduced_ecoli_ale,"position":3651293,"sequence change":"Δ2,422 bp"},
    {"experiment":reduced_ecoli_ale,"position":2173363,"sequence change":"Δ2 bp"},
    {"experiment":reduced_ecoli_ale,"position":3888432,"sequence change":"Δ18,167 bp"},
    {"experiment":reduced_ecoli_ale,"position":1962566,"sequence change":"Δ16,764 bp"},
    {"experiment":reduced_ecoli_ale,"position":2320144,"sequence change":"Δ16,489 bp"},
    {"experiment":reduced_ecoli_ale,"position":3453928,"sequence change":"Δ15,926 bp"},
    {"experiment":reduced_ecoli_ale,"position":2601201,"sequence change":"Δ13,595 bp"},
    {"experiment":reduced_ecoli_ale,"position":2165151,"sequence change":"Δ12,060 bp"},
    {"experiment":reduced_ecoli_ale,"position":1129397,"sequence change":"Δ11,591 bp"},
    {"experiment":reduced_ecoli_ale,"position":3593299,"sequence change":"Δ10 bp"},
    {"experiment":reduced_ecoli_ale,"position":3720634,"sequence change":"Δ1,442 bp"},
    {"experiment":reduced_ecoli_ale,"position":1872030,"sequence change":"Δ1,437 bp"},
    {"experiment":reduced_ecoli_ale,"position":1299499,"sequence change":"Δ1,199 bp"},
    {"experiment":reduced_ecoli_ale,"position":687860,"sequence change":"Δ1,185 bp"},
    {"experiment":reduced_ecoli_ale,"position":3581139,"sequence change":"3915 bp→6 bp"},
    {"experiment":reduced_ecoli_ale,"position":1197137,"sequence change":"25956 bp→12 bp"},
    {"experiment":reduced_ecoli_ale,"position":3110680,"sequence change":"25702 bp→GA"},
]

for mut in muts_to_remove:
    muts_df = muts_df[~(
        (muts_df["exp"] == mut["experiment"])
        & (muts_df["Position"] == mut["position"])
        & (muts_df["Sequence Change"] == mut["sequence change"])
    )]
    
# Filter the following mutations out if they come from specific experiments.
muts_df = muts_df[(~((muts_df.exp=="SER") & (muts_df["Gene"]=="cyoB") & (muts_df["Details"]=="W190* (TGG→TAG)")))]
muts_df = muts_df[(~((muts_df.exp=="SER") & (muts_df["Gene"]=="eno") & (muts_df["Details"]=="L193Q (CTG→CAG)")))]
muts_df = muts_df[(~((muts_df.exp=="SER") & (muts_df["Gene"]=="chbF") & (muts_df["Details"]=="V133E (GTA→GAA)")))]
muts_df = muts_df[(~((muts_df.exp=="SER") & (muts_df["Gene"]=="fumC") & (muts_df["Details"]=="G354G (GGT→GGC)")))]
muts_df = muts_df[(~((muts_df.exp=="SER") & (muts_df["Gene"]=="yegE") & (muts_df["Details"]=="S683Y (TCC→TAC)")))]
muts_df = muts_df[(~((muts_df.exp=="SER") & (muts_df["Gene"]=="rcsC") & (muts_df["Details"]=="L594F (TTG→TTC)")))]


muts_df = muts_df[(~((muts_df.exp=="GYD") & (muts_df["Gene"]=="fumC") & (muts_df["Details"]=="G354G (GGT→GGC)")))]
muts_df = muts_df[(~((muts_df.exp=="GYD") & (muts_df["Gene"]=="yegE") & (muts_df["Details"]=="S683Y (TCC→TAC)")))]
muts_df = muts_df[(~((muts_df.exp=="GYD") & (muts_df["Gene"]=="rcsC") & (muts_df["Details"]=="L594F (TTG→TTC)")))]


cck_exp_list = [
    "tpi",
    "pts",
    "pgi",
    "sdh",
    "gnd",
    "evo"
]

muts_df = muts_df[~(
    (muts_df.exp.isin(cck_exp_list))
    & (
        (muts_df["Gene"]=="wcaA")
        | (muts_df["Gene"]=="corA")
    )
)]  # The details of the mutations to filter out should probably include the sequence change.
# to use with filtering out mutations 
muts_df = muts_df.reset_index(drop=True)

MG1655_anchestor = muts_df[muts_df.exp=="MG1655_anchestor"].copy()
muts_df = muts_df[~(muts_df.index.isin(list(MG1655_anchestor.index)))]

muts_df = muts_df[(~((muts_df.exp=="MG1655_chloramphenicol") & (muts_df["Position"]==4390754) & (muts_df["Details"]=="A252A (GCC→GCA)")))]
muts_df = muts_df[(~((muts_df.exp=="MG1655_chloramphenicol") & (muts_df["Position"]==803662) & (muts_df["Details"]=="L54I (CTT→ATT)")))]
muts_df = muts_df[(~((muts_df.exp=="MG1655_chloramphenicol") & (muts_df["Position"]==1905761) & (muts_df["Details"]=="G25D (GGT→GAT)")))]

muts_df = muts_df[(~((muts_df.exp=="MG1655_doxycycline") & (muts_df["Position"]==4390754) & (muts_df["Details"]=="A252A (GCC→GCA)")))]
muts_df = muts_df[(~((muts_df.exp=="MG1655_doxycycline") & (muts_df["Position"]==803662) & (muts_df["Details"]=="L54I (CTT→ATT)")))]
muts_df = muts_df[(~((muts_df.exp=="MG1655_doxycycline") & (muts_df["Position"]==1905761) & (muts_df["Details"]=="G25D (GGT→GAT)")))]

muts_df = muts_df[(~((muts_df.exp=="MG1655_trimethoprim") & (muts_df["Position"]==4390754) & (muts_df["Details"]=="A252A (GCC→GCA)")))]
muts_df = muts_df[(~((muts_df.exp=="MG1655_trimethoprim") & (muts_df["Position"]==803662) & (muts_df["Details"]=="L54I (CTT→ATT)")))]
muts_df = muts_df[(~((muts_df.exp=="MG1655_trimethoprim") & (muts_df["Position"]==1905761) & (muts_df["Details"]=="G25D (GGT→GAT)")))]

# to use with filtering out mutations 
muts_df = muts_df.reset_index(drop=True)

# Remove starting strain mutations from all experiments according to each experiment's starting strain

def _get_sample_name(ale, flask, isolate, tech_rep):  # Doesn't include the experiment name
    sample_name = str(int(ale)) + " " + str(int(flask)) + " " + str(int(isolate)) + " " + str(int(tech_rep))
    return sample_name

muts_df["sample"] = muts_df.apply(lambda r: _get_sample_name(r.ale, r.flask, r.isolate, r.tech_rep), axis=1)
muts_df

muts_df = muts_df.copy()
starting_strains = muts_df[muts_df["sample"]=="0 0 1 1"]  # Need an exact match
muts_df = muts_df[muts_df["sample"]!="0 0 1 1"]  # remove starting strain samples
for e, exp_starting_strain_muts in starting_strains.groupby("exp"):
    rows_to_filter = (
        muts_df["exp"].isin(exp_starting_strain_muts["exp"])
        & muts_df["Position"].isin(exp_starting_strain_muts["Position"])
        & muts_df["Sequence Change"].isin(exp_starting_strain_muts["Sequence Change"])
        & muts_df["Details"].isin(exp_starting_strain_muts["Details"])
    )
    muts_df.drop(muts_df[rows_to_filter].index, inplace=True)  # have to ensure that muts_df indeces are unique
    
muts_df = muts_df[
    ~(
        (muts_df.exp == "GYD")
        & ((muts_df.ale == 35) | (muts_df.ale == 34))
    )
]

# This work is also currently duplicated NB4. Keep the implementation here and remove the code from NB4
muts_df["Gene"] = muts_df["Gene"].apply(lambda a: "rph" if a == "[rph], [rph]" else a)
muts_df["Gene"] = muts_df["Gene"].apply(lambda a: "rph" if a == "[rph],[rph]" else a)

muts_df = muts_df[muts_df['Details'].notna()]
muts_df["coding"] = muts_df.Details.apply(is_coding_mut)
muts_df["Gene"] = muts_df.apply(lambda row: row["Gene"].replace(", ","/") if not row["coding"] else row["Gene"], axis=1)

muts_df

,exp,ale,flask,isolate,tech_rep,presence,Position,Mutation Type,Sequence Change,Details,Gene,Reference Seq,sample,exp ALE,coding
1,tpiKO,1,28,1,1,1.00/0.00,3088193,SNP,C→A,intergenic (+333/‑91),metK/galP,NC_000913,1 28 1 1,tpiKO 1,False
3,tpiKO,1,28,2,1,1.00/0.00,3088193,SNP,C→A,intergenic (+333/‑91),metK/galP,NC_000913,1 28 2 1,tpiKO 1,False
4,tpiKO,1,28,2,1,1.00/0.00,4183214,SNP,C→T,T657I (ACC→ATC),rpoB,NC_000913,1 28 2 1,tpiKO 1,True
5,untreated,1,10,0,1,0.24/0.00,819360,SNP,G→T,L22L (CTG→CTT),moaE,NC_000913,1 10 0 1,untreated 1,True
6,untreated,1,10,0,1,0.31/0.00,4183378,SNP,T→C,S712P (TCC→CCC),rpoB,NC_000913,1 10 0 1,untreated 1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111377,TOL_isobutyric_acid,8,50,3,1,1.00/0.00,3725175,SNP,C→G,E48D (GAG→GAC),glyQ,NC_000913,8 50 3 1,TOL_isobutyric_acid 8,True
111378,TOL_isobutyric_acid,8,50,3,1,1.00/0.00,2563402,SNP,C→A,intergenic (+285/‑190),yffP/yffQ,NC_000913,8 50 3 1,TOL_isobutyric_acid 8,False
111379,TOL_isobutyric_acid,8,50,3,1,1.00/0.00,3851044,SNP,T→G,N17H (AAC→CAC),ilvN,NC_000913,8 50 3 1,TOL_isobutyric_acid 8,True
111380,TOL_isobutyric_acid,8,50,3,1,1.00/0.00,1978503,DEL,Δ776 bp,,"insB1, insA",NC_000913,8 50 3 1,TOL_isobutyric_acid 8,True


## Remove Hypermutators

In [12]:
# to normalize clones and populations, only keeping mutations with 100% frequency
print(len(muts_df))
clone_pop_norm_muts_df = muts_df[muts_df["presence"].str.contains('1.00')]
print(len(clone_pop_norm_muts_df))

81539
61152


In [13]:
# Just checking the endpoint flasks for now. Currently assuming that hypermutators will always fix in an ALE.
# If this assumption proves inaccurate, can set endpoint_flask_only=False,though will take much longer.
ep_max_freq_mut_df = get_multi_exp_max_freq_mut_df(clone_pop_norm_muts_df, endpoint_flask_only=True)

In [14]:
# HYPERMUTATOR_GENES contains all synonyms for genes known to manifest hypermutator phenotypes after being mutated.
# This list of genes is from 10.1016/S0966-842X(98)01424-3 and synonyms were retrieved from ecocyc.
HYPERMUTATOR_GENES = {
    "mutD", "dnaQ", "mutS", "ant", "plm", "fdv",
    "mutL", "mutH", "mutR", "topB", "prv", 'uvrD',
    "uvr502", "srjC", "uvrE", "dar-2", "dda",
    "mutU", "pdeB", "rad", "recL", "mutM",
    "fpg", "mutY", "micA", "mutT", "nudA",
    "dnaE", "polC", "sdgC", 'polA', "resA",
    "mutA", "glyV", "mutC", "glyW", "ins",
    "dam", "miaA", "trpX", "sodA", "sodB",
    "oxyR"  # We've had this one mutate in ALE experiments and not cause hypermutation, so don't include unless also checking if ALE is an outlier with mutation counts
    "nth", "nei", "xthA", "xth", "nfo",
    "ung", "vsr", "ada", "ogt", "recA",
    "zab", "umuB", "tif", "lexB", "recH", "rnmB", "srf",
    "recG", "radC", "spoV", "ssb", "exrB", "lexC"
    "hns"  # We've had this one mutate in ALE experiments and not cause hypermutation, so don't include unless also checking if ALE is an outlier with mutation counts
}


# def _get_mutated_hypermutator_genes(mutated_genes_str):  # currently checking annotation that could contain an incomplete set of affected genes. Would be more accurate to use positions.
def _get_mutated_hypermutator_genes(m):
    mutated_hypermutator_genes = set()
    if m["coding"]: # Mutation affect a gene's nucleotides
        mutated_genes_str = m["Gene"]
        mutated_genes = set(get_clean_mut_gene_list(mutated_genes_str))
        mutated_hypermutator_genes = mutated_genes & HYPERMUTATOR_GENES
    return mutated_hypermutator_genes


assert(_get_mutated_hypermutator_genes({"coding": True, "Gene": "asdf, zxcv"}) == set())
assert(_get_mutated_hypermutator_genes({"coding": False, "Gene": "asdf, zxcv, mutL"}) == set())
assert(_get_mutated_hypermutator_genes({"coding": True, "Gene": "asdf, zxcv, mutL"}) == {"mutL"})
assert(_get_mutated_hypermutator_genes({"coding": True, "Gene": "mutA, asdf, zxcv, mutL"}) == {"mutA", "mutL"})
assert(_get_mutated_hypermutator_genes({"coding": True, "Gene": "asdf, zxcv, insB-5"}) == set())  # "ins" is a hypermutator genes; ensure that substrings aren't being identified as hypermutator genes with "insB-5".
assert(_get_mutated_hypermutator_genes({"coding": True, "Gene": '[ynaJ],uspE,fnr,ogt,abgT,abgB,abgA,abgR,mcaS,smrA,dgcM,zntB,fnrS,ynaL,dbpA,[ttcA]'}) == {"ogt"})  # testing complicated annotation string

ep_max_freq_mut_df["hypermutator gene hit"] = ep_max_freq_mut_df.apply(lambda m: ', '.join(_get_mutated_hypermutator_genes(m)), axis=1)

In [15]:
# For use with building distribution to find outliers
exp_ale_mut_cnt_df_rows = []
for exp, exp_mut_df in ep_max_freq_mut_df.groupby("exp"):
    for exp_ale, ale_df in exp_mut_df.groupby("exp ALE"):
        exp_ale_mut_cnt_df_rows.append({"experiment": exp, "exp ALE": exp_ale, "mutation count": len(ale_df)})
exp_ale_mut_cnt_df = pd.DataFrame(exp_ale_mut_cnt_df_rows)   

In [18]:
outlier_mut_cnts = boxplot_stats(exp_ale_mut_cnt_df["mutation count"]).pop(0)['fliers']  # should return something about outliers, though I forget what

outlier_mut_count_ALEs = set(exp_ale_mut_cnt_df[exp_ale_mut_cnt_df["mutation count"].isin(outlier_mut_cnts)]["exp ALE"])
ALEs_with_hyp_mut_gene = set(ep_max_freq_mut_df[ep_max_freq_mut_df["hypermutator gene hit"]!='']["exp ALE"].unique())
pred_hyp_mut_ALEs = ALEs_with_hyp_mut_gene & outlier_mut_count_ALEs

ep_max_freq_mut_df["predicted hypermutator ALE"] = ep_max_freq_mut_df.apply(lambda m: m["exp ALE"] in pred_hyp_mut_ALEs, axis=1)
all_hyp_mut_ALEs = ep_max_freq_mut_df[ep_max_freq_mut_df["predicted hypermutator ALE"]]["exp ALE"].unique()

exp_hypermut_ale_frac_df_rows = []
for exp, exp_mut_df in ep_max_freq_mut_df.groupby("exp"):
    exp_outlier_ale_fraction = 0
    exp_ale_cnt = len(exp_mut_df["exp ALE"].unique())
    exp_hyp_ale_cnt = len(exp_mut_df[exp_mut_df["predicted hypermutator ALE"]]["exp ALE"].unique())
    if exp_hyp_ale_cnt > 0:
        exp_outlier_ale_fraction = exp_hyp_ale_cnt / exp_ale_cnt
    exp_hypermut_ale_frac_df_rows.append({"experiment": exp, "experiment ALE hypermutator fraction": exp_outlier_ale_fraction})
exp_hypermut_ale_frac_df = pd.DataFrame(exp_hypermut_ale_frac_df_rows)

exp_hypermut_ale_frac_df[exp_hypermut_ale_frac_df["experiment ALE hypermutator fraction"]!=0]

,experiment,experiment ALE hypermutator fraction
0,42C,0.100000
6,BOP27-M9-NC_000913_3gb-log-37-Glucose2-NH4Cl1-...,0.250000
13,C321_A_earlyfix,0.428571
23,E12,1.000000
24,E14,1.000000
25,E19,1.000000
26,E2,1.000000
27,E20,1.000000
28,E5,1.000000
29,E9,1.000000


In [21]:
muts_df = muts_df[~(muts_df["exp ALE"].isin(all_hyp_mut_ALEs))]  # removing hypermutator ALEs for output

## Get Unique High Presence Mutations

In [22]:
print(len(muts_df))
muts_df = muts_df[muts_df['presence'].str.split('/').apply(lambda t: float(t[0])) >= 0.5]
print(len(muts_df))

41602
22191


In [23]:
# Just get ALE-unique mutations. Do as much mutation filtering as possible running this to cut down the runtime on the finding ALE-unique mutations.
print(len(muts_df))
muts_df = get_multi_exp_max_freq_mut_df(muts_df, endpoint_flask_only=False)
muts_df

22191


,exp,ale,flask,isolate,tech_rep,presence,Position,Mutation Type,Sequence Change,Details,Gene,Reference Seq,sample,exp ALE,coding
82393,42C,1,124,1,1,1.00/0.00,702352,DEL,Δ21 bp,coding (380‑400/1149 nt),nagA,NC_000913,1 124 1 1,42C 1,True
82394,42C,1,124,1,1,1.00/0.00,1308318,SNP,G→C,R110G (CGT→GGT),clsA,NC_000913,1 124 1 1,42C 1,True
82392,42C,1,124,1,1,1.00/0.00,2173364,DEL,Δ1 bp,intergenic (‑2/+1),gatC/gatC,NC_000913,1 124 1 1,42C 1,False
82395,42C,1,124,1,1,1.00/0.00,3815859,DEL,Δ82 bp,,rph,NC_000913,1 124 1 1,42C 1,True
82396,42C,1,124,1,1,1.00/0.00,4187550,SNP,C→T,A734V (GCG→GTG),rpoC,NC_000913,1 124 1 1,42C 1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12276,wt,2,1,1,1,1.00/0.00,3291610,MOB,IS3 (–) +5 bp :: +T,coding (270‑274/1092 nt),yraK,NC_000913,2 1 1 1,wt 2,True
12277,wt,2,1,1,1,1.00/0.00,3471736,SNP,A→T,F593L (TTT→TTA),fusA,NC_000913,2 1 1 1,wt 2,True
12278,wt,2,1,1,1,1.00/0.00,3917504,DEL,Δ28 bp,coding (735‑762/864 nt),atpG,NC_000913,2 1 1 1,wt 2,True
12279,wt,2,1,1,1,1.00/0.00,3968005,DEL,(A)7→6,coding (90/1104 nt),rfe,NC_000913,2 1 1 1,wt 2,True


## Save Curated SNPs

Want to focus on SNPs because mapping deletions from reference strain to other strains is fraught

In [101]:
muts_df = muts_df[muts_df['Mutation Type'] == 'SNP']
# make sure the ALE IDs will be strings
muts_df.index = map(lambda int_id: f'ale_{int_id}', muts_df.index)
muts_df.to_csv(Path(DATA_DIR, 'mutations_curated.csv'))

# Load Bitome Annotations

All of these files are provided as part of this repository

## Bitome Object

In [32]:
# origin from GenBank annotation
# terminus from 1. Duggin, I. G. & Bell, S. D. J. Mol. Biol. (2009). with the following terA/terC sequences:
# ter_a = 'AATTAGTATGTTGTAACTAAAGT'
# ter_c = 'ATATAGGATGTTGTAACTAATAT'
mg1655 = Bitome(
    Path(BITOME_DATA_PATH, 'NC_000913.3.gb'),
    name='MG1655',
    origin=(3925743, 3925975), terminus=(1341745, 1609180),
    gene_table=Path(BITOME_DATA_PATH, 'gene_info_supp.csv'),
    tu_table=Path(BITOME_DATA_PATH, 'tu.csv'),
    operon_table=Path(BITOME_DATA_PATH, 'operon.csv'),
    tss_table=Path(BITOME_DATA_PATH, 'tss.csv'),
    tfbs_table=Path(BITOME_DATA_PATH, 'tfbs.csv'),
    terminator_table=Path(BITOME_DATA_PATH, 'terminator.csv'),
    attenuator_table=Path(BITOME_DATA_PATH, 'attenuator.csv'),
    rbs_table=Path(BITOME_DATA_PATH, 'rbs.csv')
)

## TSS Table and Regulatory Interactions

In [34]:
bitome_mg1655_tss_df = pd.read_csv(Path(BITOME_DATA_PATH, 'tss_combined.csv'), index_col=0)
bitome_mg1655_tss_df.head()

,tss,strand,tss_base,promoter_id,tu_id,tu_name,gene_ids,gene_names,n_genes,tu_left,...,Sigma32_box_35_score,Sigma28_box_10_score,Sigma28_box_35_score,Sigma24_box_10_score,Sigma24_box_35_score,box_10_seq,box_35_seq,spacer_len,spacer_gc,replication_region
tss_recon_id,,,,,,,,,,,,,,,,,,,,,
TSS_1832013_r,1832013,-1,A,ECK120009851,ECK120008913,astCADBE,b1744;b1745;b1746;b1747;b1748,astE;astB;astD;astA;astC,5,1825955.0,...,-inf,NaN,NaN,NaN,-0.763211,TATTAT,CTACAT,0.0,NaN,leading
TSS_1832044_r,1832044,-1,A,ECK120009852,ECK120008914,astCADBE,b1744;b1745;b1746;b1747;b1748,astE;astB;astD;astA;astC,5,1825955.0,...,0.910541,NaN,NaN,-0.447979,-10.391454,TGCAA,TGGCAC,0.0,NaN,leading
TSS_1832039_r,1832039,-1,G,ECK120009853,ECK120008915,astCADBE,b1744;b1745;b1746;b1747;b1748,astE;astB;astD;astA;astC,5,1825955.0,...,-3.223795,NaN,NaN,NaN,-10.004002,CAATCTACATT,CTGGCT,0.0,NaN,leading
TSS_2800656_f,2800656,1,A,ECK120009855,ECK120008916,nrdHIEF,b2673;b2674;b2675;b2676,nrdH;nrdI;nrdE;nrdF,4,2800656.0,...,7.898709,NaN,NaN,NaN,-10.351926,TAGTAT,TTGAAT,0.0,NaN,lagging
TSS_4105785_f,4105785,1,G,ECK120010879,ECK120008917,cpxPQ,b4484,cpxP,1,4105785.0,...,0.464915,NaN,NaN,NaN,-10.821138,TAAACT,CTGACG,0.0,NaN,leading


In [25]:
ri_df = pd.read_csv(Path(BITOME_DATA_PATH, 'regulatory_interaction.csv'), index_col=0)
ri_df.head()

,REGULATORY_INTERACTION_ID,CONFORMATION_ID,PROMOTER_ID,SITE_ID,RI_FUNCTION,CENTER_POSITION,RI_DIST_FIRST_GENE,RI_FIRST_GENE_ID,REGULATORY_INTERACTION_NOTE,RI_SEQUENCE,...,SITE_NOTE,SITE_LENGTH,SITE_ABSOLUTE_CENTER_POSITION,SITE_SEQUENCE_REVERSE_COMPLEMENT,TF_ID,T_FACTOR_ID,T_FACTOR_NAME,T_FACTOR_SITE_LENGTH,T_FACTOR_KEY_ID_ORG,T_FACTOR_SITE_GROUP
0,ECK120030767,ECK125269234,ECK120010130,ECK120012046,activator,-61.5,-99.5,ECK120000520,The CRP-binding site controlling the <i>lacZ</...,gcaacgcaatTAATGTGAGTTAGCTCACTCATtaggcacccc,...,NaN,22.0,366404.5,gcaacgcaatTAATGTGAGTTAGCTCACTCATtaggcacccc,29,ECK120011235,CRP,22,ECK12,291
1,ECK120030768,ECK125269282,ECK120010108,ECK120012328,activator,-64.0,-91.0,ECK120000049,NaN,tgctatgccaTAGCATTTTTATCCATAagattagcgg,...,NaN,17.0,70139.0,tgctatgccaTAGCATTTTTATCCATAagattagcgg,14,ECK120012316,AraC,17,ECK12,16
2,ECK120030769,ECK125269395,ECK120010108,ECK120012328,repressor,-64.0,-91.0,ECK120000049,NaN,tgctatgccaTAGCATTTTTATCCATAagattagcgg,...,NaN,17.0,70139.0,tgctatgccaTAGCATTTTTATCCATAagattagcgg,14,ECK120012316,AraC,17,ECK12,16
3,ECK120030770,ECK125269282,ECK120010988,ECK120012328,activator,-84.0,-248.0,ECK120000050,NaN,ccgctaatctTATGGATAAAAATGCTAtggcatagca,...,NaN,17.0,70139.0,tgctatgccaTAGCATTTTTATCCATAagattagcgg,14,ECK120012316,AraC,17,ECK12,16
4,ECK120030771,ECK125269395,ECK120010988,ECK120012328,repressor,-84.0,-248.0,ECK120000050,NaN,ccgctaatctTATGGATAAAAATGCTAtggcatagca,...,NaN,17.0,70139.0,tgctatgccaTAGCATTTTTATCCATAagattagcgg,14,ECK120012316,AraC,17,ECK12,16


# Extract Upstream Regions from MG1655

Here we will extract all of the MG1655 regions we wish to BLAST across all genomes

Here we can focus on the first gene transcribed by a given TSS and the 5' UTR that defines, plus some distance upstream of the TSS to capture the promoter region

In [161]:
# Define range constants (make sure that if you change these here you also change them in `1__analyze_alleleome`)
FIVE_UTR_TSS_UP = 200
FIVE_UTR_GENE_DOWN = 50

# so just the 5' UTR length varies which will create different search ranges - because we also take a constant up from TSS
# TFBS are interesting and can "break the bank" if they're more upstream than the extended promoter
# we can seed the extended promoter length and add the 5' UTR
bnum_to_up_from_gene_start = {}

# also want to keep track of all ranges of sequence that we are extracting
mg_extract_ranges = []

tss_with_gene_ids = bitome_mg1655_tss_df[bitome_mg1655_tss_df['gene_ids'].notna()]

# EXCEPTIONS TO IGNORE - dinG and wza both have TSSes that are allegedly downstream of the gene start
# which is clearly not right - just ignore for now - makes 5' UTR look mega huge as it goes thru a whole gene
# also exclude narWV, which has an insanely long 5' UTR for some reason
tu_exceptions = ['wzbc', 'dinG-ybiB', 'narWV']
tss_with_gene_ids = tss_with_gene_ids[~tss_with_gene_ids['tu_name'].isin(tu_exceptions)]

gene_ids_seen = []

five_utr_seqs = []

for tss_id, tss_row in tss_with_gene_ids.iterrows():

    tss_strand = tss_row['strand']

    # identify the gene closest to this TSS
    tss_gene_df = mg1655.gene_table.loc[tss_row['gene_ids'].split(';')]
    def dist_to_tss(gene_row):
        if tss_strand == 1:
            return gene_row['left'] - tss_row['tss']
        else:
            return tss_row['tss'] - gene_row['right']
    tss_gene_df = tss_gene_df.assign(
        tss_dist=tss_gene_df.apply(dist_to_tss, axis=1)
    )
    tss_gene_df = tss_gene_df.sort_values(by='tss_dist')
    closest_gene_row = tss_gene_df.iloc[0]
    closest_gene_id = closest_gene_row.name
    # update the upstream from gene start distance that we will use later
    if closest_gene_id in bnum_to_up_from_gene_start:
        bnum_to_up_from_gene_start[closest_gene_id] = max(bnum_to_up_from_gene_start[closest_gene_id], closest_gene_row['tss_dist'])
    else:
        bnum_to_up_from_gene_start[closest_gene_id] = closest_gene_row['tss_dist']
    
    if tss_strand == 1:
        gene_start = closest_gene_row['left']
        five_utr_l, five_utr_r = tss_row['tss'] - FIVE_UTR_TSS_UP, gene_start + FIVE_UTR_GENE_DOWN
    else:
        gene_start = closest_gene_row['right']
        five_utr_l, five_utr_r = gene_start - FIVE_UTR_GENE_DOWN, tss_row['tss'] + FIVE_UTR_TSS_UP
        
    # always do the 5' UTR because different TSSes will have more/less to TSS (some overlap though)
    five_utr_seq = str(mg1655.get_sequence(five_utr_l, five_utr_r, tss_strand))
    mg_extract_ranges.append((five_utr_l, five_utr_r))
    five_utr_seqs.append((f'{tss_id}__{closest_gene_id}__5utr', five_utr_seq))
            
write_fasta_lines(five_utr_seqs, Path(BLAST_INPUT_DIR, 'mg1655_upstream_regions.fasta'))

# we can systematically add a constant amount to the 5' UTR dist from gene ups
bnum_to_up_from_gene_start = {k: v + FIVE_UTR_TSS_UP for k, v in bnum_to_up_from_gene_start.items()}

# BLAST for MG1655 Features across Pangenome

## Read in Pangenome Sequences and Parsed GFF Feature Tables

Want to create a database per E. coli b-number

In [22]:
genome_id_to_seq_dict = {}
genome_id_to_feat_table = {}

for i, (_, valid_file_row) in enumerate(valid_genome_md.iterrows()):

    print(f"{i+1}/{valid_genome_md.shape[0]}", end='\r')
    genome_id = valid_file_row['genome_id']
    genome_id_to_seq_dict[genome_id] = load_sequences_from_fasta(valid_file_row['fna'])
    genome_id_to_feat_table[genome_id] = pd.read_csv(
        Path(Path(valid_file_row['fna']).parent, 'feature_table.csv'),
        index_col=0
    )

## Create Databases and Run BLAST

This step will call BLAST in the command line - so BLAST must be installed - may take a few hours to run

In [131]:
GENE_START_DOWN_PAD = 50
GENE_START_DOWN = FIVE_UTR_GENE_DOWN + GENE_START_DOWN_PAD
# to account for gaps/indels
GENE_START_UP_PAD = 100
GENE_START_UP_DEFAULT = np.mean([len(tup[1]) for tup in five_utr_seqs]) + FIVE_UTR_GENE_DOWN + GENE_START_UP_PAD

# don't use valid file table index, dumb float issue with IDs
# make a BLAST DB for this genome's sequence
# we only want to consider regions upstream/downstream of genes
# this will help us narrow down proper hits a bit later
failed_genome_ids = []
for i, (_, valid_file_row) in enumerate(valid_genome_md.iterrows()):
    print(f'Genome {i+1}/{valid_genome_md.shape[0]}', end='\r')
    try:
        genome_id = valid_file_row['genome_id']
        genome_blast_dir = Path(PANGENOME_DIR, genome_id, 'blast')
        genome_blast_dir.mkdir(exist_ok=True)

        # get the set of sequences for this genome
        genome_seq_dict = genome_id_to_seq_dict[genome_id]

        # only consider CDS upstreams/downstreams
        genome_feat_table = genome_id_to_feat_table[genome_id]
        genome_feat_table = genome_feat_table[genome_feat_table['feature_type'] == 'CDS']

        search_regions = []
        for _, cds_row in genome_feat_table.iterrows():

            # can skip this gene if it's not in our reverse bnumber lookup - that means 
            # it's not in a MG1655 gene cluster and thus we can't have a known MG feature upstream of it
            if cds_row['ID'] not in pan_gene_to_bnums:
                continue
            else:
                bnums_for_pan_gene = pan_gene_to_bnums[cds_row['ID']]

            # figure out how far upstream from gene start we have features in MG
            gene_up_dist_mg = max([bnum_to_up_from_gene_start.get(bnum, 0) for bnum in bnums_for_pan_gene])
            if gene_up_dist_mg == 0:
                gene_start_up_pad = GENE_START_UP_DEFAULT
            else:
                gene_start_up_pad = gene_up_dist_mg + GENE_START_UP_PAD

            # now we have all the info we need to extract the right sequence from this genome
            cds_l, cds_r = cds_row[['left', 'right']]
            cds_strand_int = 1 if cds_row['strand'] == '+' else -1

            # define the up ranges depending on strand
            if cds_strand_int == 1:
                up_l, up_r = cds_l - gene_start_up_pad, cds_l + GENE_START_DOWN
            else:
                up_l, up_r = cds_r - GENE_START_DOWN, cds_r + gene_start_up_pad

            cds_chrom_plasm_id = cds_row['sequence_id'].replace('accn|', '')
            relevant_seq = [seq for seq_id, seq in genome_seq_dict.items() if cds_chrom_plasm_id in seq_id][0]

            # extract the up and down seqs - get_sequence works under the hood to handle overhangs after subtraction
            up_seq = get_sequence(relevant_seq, up_l, up_r, cds_strand_int).upper()

            # add these to our running list of database search regions for this genome
            # include informative IDs with the CDS ID (includes genome ID, the chrom_plasm_id, and the bnumbers this feature matches with)
            base_id = f"{cds_row['ID']}__{cds_chrom_plasm_id}"
            search_regions.append((f'{base_id}__up', up_seq))

        # write all of these search regions to a FASTA
        db_fasta = Path(genome_blast_dir, f'{genome_id}_blastdb.fasta')
        write_fasta_lines(search_regions, db_fasta)

        # now we can actually create a BLAST database for this region
        db_dir = Path(genome_blast_dir, f'{genome_id}_blastdb')
        subprocess.check_output(['makeblastdb',
                                 '-in', db_fasta, '-out', db_dir,
                                 '-parse_seqids', '-dbtype', 'nucl'])

        # and now we can actually just fire off the BLAST!
        subprocess.check_output(['blastn',
                                 '-task', 'megablast',
                                 '-db', db_dir,
                                 '-query', Path(BLAST_INPUT_DIR, 'mg1655_upstream_regions.fasta'),
                                 '-out', Path(genome_blast_dir, f'blast_result_raw.csv'),
                                 '-evalue', '10',
                                 '-outfmt', '10',
                                 '-num_threads', '8'])
    except:
        failed_genome_ids.append(valid_file_row['fna'].split('/')[-1].replace('.fna', ''))
    
print(failed_genome_ids)

[]nome 2350/2350


## Read in MG1655 Sequences

In [40]:
mg1655_feat_id_to_seqs = load_sequences_from_fasta(
    Path(BLAST_INPUT_DIR, 'mg1655_upstream_regions.fasta')
)

## Parse and QC BLAST Results

In [133]:
BLAST_COL_NAMES = [
    'bitome_feat_id', 'pangenome_gene_id', 'percent', 'align_length', 'n_mismatch', 'n_gapopen',
    'bitome_feat_start', 'bitome_feat_end', 'genome_start', 'genome_end', 'e_value', 'bit_score'  
]

# move through the genomes and parse the BLAST results
for i, (_, valid_file_row) in enumerate(valid_genome_md.iterrows()):
    
    genome_id = valid_file_row['genome_id']
    genome_blast_dir = Path(PANGENOME_DIR, genome_id, 'blast')

    print(f'Genome {i+1}/{valid_genome_md.shape[0]}: {genome_id}', end='\r')
    # read in the blast result dataframe
    blast_result_df = pd.read_csv(
        Path(genome_blast_dir, 'blast_result_raw.csv'),
        header=None, names=BLAST_COL_NAMES
    )
    
    # weirdly ended up with : instead of | after fig - roll with it for now
    # fix pangenome feature ID and add chrom plasm
    blast_result_df = blast_result_df.assign(
        pan_gene_full_id=blast_result_df['pangenome_gene_id'].str.replace(':', '|'),
        feature_id=blast_result_df['pangenome_gene_id'].str.replace(':', '|').apply(
            lambda pgid: pgid.split('__')[0]
        )
    )
    blast_result_df = blast_result_df.assign(
        chrom_plasm_id=blast_result_df['pangenome_gene_id'].apply(
            lambda pgid: pgid.split('__')[1]
        )
    )
    
    # add in the genome ID; have to do dumb thing because of float issue
    blast_result_df = blast_result_df.assign(
        genome_id=blast_result_df['feature_id'].apply(
            lambda feat_id: f"delete{feat_id.split('|')[1].split('.peg.')[0]}"
        )
    )
    
    # read in the database seqs for this genome ID; get the seq each match is from
    genome_db_seq_dict = load_sequences_from_fasta(Path(genome_blast_dir, f'{genome_id}_blastdb.fasta'))
    
    # this assumes that the matches are within the confines of a DB sequence, which must be the case really
    # no weird "wraparounds"
    # NOTE: since we already handled strand when we extracted sequences for the DB, we do NOT have to worry
    # about it here
    def get_seq_for_blast_result(blast_result_row):
        # get sequence of the matched feature
        relevant_seq = genome_db_seq_dict[blast_result_row['pan_gene_full_id']]
        seq_start, seq_end = blast_result_row['genome_start'], blast_result_row['genome_end']
        return relevant_seq[seq_start-1:seq_end].upper()
    
    blast_result_df = blast_result_df.assign(
        seq=blast_result_df.apply(get_seq_for_blast_result, axis=1),
        mg_seq_len=blast_result_df['bitome_feat_id'].apply(
            lambda bfi: len(mg1655_feat_id_to_seqs[bfi])
        )
    )
    
    # now we want to make sure we keep the blast results that make sense
    # group by bitome feature ID - then drop anything that doesn't match with the expected bnum
    rows_to_keep = []
    for bitome_feat_id, bf_blast_result_df in blast_result_df.groupby('bitome_feat_id'):
        bitome_bnum = bitome_feat_id.split('__')[1]
        bf_blast_with_bnum = bf_blast_result_df[
            bf_blast_result_df['feature_id'].apply(lambda fi: fi in bnum_to_genes.get(bitome_bnum, []))
        ]
        # if we still have multiple matches, take the best by e-value
        if bf_blast_with_bnum.shape[0] > 1:
            rows_to_keep.append(bf_blast_with_bnum.sort_values(by='e_value').iloc[0])
        # just the one result, grab the row; no else to ignore empty case
        elif bf_blast_with_bnum.shape[0] == 1:
            rows_to_keep.append(bf_blast_with_bnum.iloc[0])
        
    # write this QC'ed result to file
    blast_result_qc_df = pd.DataFrame(rows_to_keep)
    blast_result_qc_df.to_csv(Path(genome_blast_dir, 'blast_result_qc.csv'))

## Combine BLAST Results by Upstream Region

In [134]:
all_blast_qc_dfs = []

for i, (_, valid_file_row) in enumerate(valid_genome_md.iterrows()):
    print(f"{i+1}/{valid_genome_md.shape[0]}", end='\r')
    genome_id = valid_file_row['genome_id']
    genome_blast_dir = Path(PANGENOME_DIR, genome_id, 'blast')
    blast_result_qc_df = pd.read_csv(Path(genome_blast_dir, 'blast_result_qc.csv'), index_col=0)
    all_blast_qc_dfs.append(blast_result_qc_df)
    
print()

all_pangenome_blast_result_df = pd.concat(all_blast_qc_dfs)
for i, (bitome_feat_id, bf_full_blast_result_df) in enumerate(all_pangenome_blast_result_df.groupby('bitome_feat_id')):
    
    print(f'{i}/~6000', end='\r')

    bitome_feat_type = bitome_feat_id.split('__')[2]
    result_folder = Path(BLAST_BY_UPSTREAM_REGION_DIR, bitome_feat_type)
    result_folder.mkdir(exist_ok=True)
    
    # write the accumulated results for this feature to the appropriate folder
    bf_full_blast_result_df.to_csv(Path(result_folder, f'{bitome_feat_id}.csv'))

2350/2350


# Align Upstream Region Alleles

This code block calls out to MUSCLE in the command line - this may take a few hours to run

In [135]:
def align_seqs_in_df(df):

    temp_align_file = Path(ALIGN_TEMP_DIR, 'temp_align_input.fasta')
    write_fasta_lines(list(enumerate(df['seq'])), Path(ALIGN_TEMP_DIR, 'temp_align_input.fasta'))

    # run MUSCLE to align sequences
    align_output_file = Path(ALIGN_TEMP_DIR, 'temp_align_output.fasta')
    # MUSCLE is throwing this abort trap error - but seemingly after writing the align file - so I'm just gonna catch it
    try:
        subprocess.check_output(['muscle', '-align', temp_align_file, '-output', align_output_file])
    except subprocess.CalledProcessError as e:
        if 'Signals.SIGABRT: 6' not in str(e):
            raise subprocess.CalledProcessError('muscle', 'OK MUSCLE actually had an unexpected problem')

    # read in the alignment back into our seq dataframe
    # NOTE THAT MUSCLE WILL RE-ORDER THE SEQUENCES
    seqs_aligned_dict = load_sequences_from_fasta(align_output_file)
    seqs_aligned_ordered = [tup[1] for tup in sorted(seqs_aligned_dict.items(), key=lambda tup: tup[0])]
    df = df.assign(
        seq_align=seqs_aligned_ordered
    )
    return df

five_utr_dir = Path(BLAST_BY_UPSTREAM_REGION_DIR, '5utr')

for i, five_utr_file in enumerate(os.listdir(five_utr_dir)):
    
    # skip if we already aligned this feature ID
    if 'align' in five_utr_file:
        continue
    
    print(f"{i+1}/{len(os.listdir(five_utr_dir))}", end='\r')

    # wonky load to avoid dumb trimming of 0s after PATRIC
    feat_result_df = pd.read_csv(Path(five_utr_dir, five_utr_file), index_col=0)
    feat_result_df = feat_result_df.assign(
        genome_id=feat_result_df['genome_id'].str.replace('delete', '')
    )
    
    feat_id = five_utr_file.split('.')[0]

    seq_counts = feat_result_df['seq'].value_counts()
    seq_counts_df = pd.DataFrame(data={'seq': seq_counts.index, 'count': seq_counts.values})
    # add in the example sequence at the top; it's likely the same as many, but can't assume that
    # want to make sure we get the aligned version of it
    mg_feat_seq = mg1655_feat_id_to_seqs[feat_id]
    seq_counts_df = pd.concat([pd.DataFrame(data={'seq': mg_feat_seq, 'count': 1}, index=['blah']), seq_counts_df])
    # re-index to ensure we have nice clean integer index to allow easy matching after MUSCLE
    seq_counts_df = seq_counts_df.reindex()
    # create the MSA (uses MUSCLE under the hood)
    seq_counts_df = align_seqs_in_df(seq_counts_df)
    
    # re-separate the example aligned seq
    example_aligned_seq = seq_counts_df.iloc[0]['seq_align']
    seq_counts_df = seq_counts_df.iloc[1:]

    def n_mismatch(seq):
        count = 0
        for b, b_examp in zip(seq, example_aligned_seq):
            if b != b_examp:
                count += 1
        return count
    seq_counts_df = seq_counts_df.assign(
        dist=seq_counts_df['seq_align'].apply(n_mismatch)
    )
    seq_counts_df = seq_counts_df.sort_values(by='count', ascending=False).reset_index()
    seq_counts_df = seq_counts_df.drop(columns='index')
    
    seq_counts_df.to_csv(Path(five_utr_dir, f'{feat_id}_align.csv'))


muscle 5.2.osxarm64 [6c6011]  25.8Gb RAM, 8 cores
Built Apr  4 2023 10:46:17
(C) Copyright 2004-2021 Robert C. Edgar.
https://drive5.com

Input: 144 seqs, avg length 281, max 295

00:00 12Mb   CPU has 8 cores, running 8 threads
00:10 248Mb   100.0% Calc posteriors 
00:11 320Mb   100.0% Consistency (1/2) 
00:11 326Mb   100.0% Consistency (2/2) 
00:11 326Mb   100.0% UPGMA5           
00:12 334Mb   100.0% Refining

muscle 5.2.osxarm64 [6c6011]  25.8Gb RAM, 8 cores
Built Apr  4 2023 10:46:17
(C) Copyright 2004-2021 Robert C. Edgar.
https://drive5.com

Input: 39 seqs, avg length 290, max 297

00:00 11Mb   CPU has 8 cores, running 8 threads


00:01 128Mb   100.0% Calc posteriors
00:01 133Mb   100.0% Consistency (1/2)
00:01 133Mb   100.0% Consistency (2/2)
00:01 133Mb   100.0% UPGMA5           
00:01 136Mb   100.0% Refining

muscle 5.2.osxarm64 [6c6011]  25.8Gb RAM, 8 cores
Built Apr  4 2023 10:46:17
(C) Copyright 2004-2021 Robert C. Edgar.
https://drive5.com

Input: 115 seqs, avg length 290, max 294

00:00 11Mb   CPU has 8 cores, running 8 threads


00:06 241Mb   100.0% Calc posteriors
00:07 285Mb   100.0% Consistency (1/2)
00:07 285Mb   100.0% Consistency (2/2)
00:07 285Mb   100.0% UPGMA5           
00:07 292Mb   100.0% Refining

muscle 5.2.osxarm64 [6c6011]  25.8Gb RAM, 8 cores
Built Apr  4 2023 10:46:17
(C) Copyright 2004-2021 Robert C. Edgar.
https://drive5.com

Input: 42 seqs, avg length 263, max 272

00:00 10Mb   CPU has 8 cores, running 8 threads


00:01 109Mb   100.0% Calc posteriors
00:01 115Mb   100.0% Consistency (1/2)
00:01 115Mb   100.0% Consistency (2/2)
00:01 115Mb   100.0% UPGMA5           
00:01 117Mb   100.0% Refining

muscle 5.2.osxarm64 [6c6011]  25.8Gb RAM, 8 cores
Built Apr  4 2023 10:46:17
(C) Copyright 2004-2021 Robert C. Edgar.
https://drive5.com

Input: 68 seqs, avg length 343, max 348

00:00 10Mb   CPU has 8 cores, running 8 threads


00:04 205Mb   100.0% Calc posteriors
00:04 221Mb   100.0% Consistency (1/2)
00:04 221Mb   100.0% Consistency (2/2)
00:04 221Mb   100.0% UPGMA5           
00:04 225Mb   100.0% Refining

muscle 5.2.osxarm64 [6c6011]  25.8Gb RAM, 8 cores
Built Apr  4 2023 10:46:17
(C) Copyright 2004-2021 Robert C. Edgar.
https://drive5.com

Input: 106 seqs, avg length 411, max 415

00:00 11Mb   CPU has 8 cores, running 8 threads


...

00:04 504Mb   100.0% Calc posteriors
00:04 517Mb   100.0% Consistency (1/2)
00:04 518Mb   100.0% Consistency (2/2)
00:04 518Mb   100.0% UPGMA5           
00:05 524Mb   100.0% Refining

muscle 5.2.osxarm64 [6c6011]  25.8Gb RAM, 8 cores
Built Apr  4 2023 10:46:17
(C) Copyright 2004-2021 Robert C. Edgar.
https://drive5.com

Input: 198 seqs, avg length 308, max 319

00:00 12Mb   CPU has 8 cores, running 8 threads


00:23 920Mb   100.0% Calc posteriors 
00:24 1.1Gb   100.0% Consistency (1/2) 
00:25 1.1Gb   100.0% Consistency (2/2) 
00:25 1.1Gb   100.0% UPGMA5           
00:27 1.1Gb   100.0% Refining



muscle 5.2.osxarm64 [6c6011]  25.8Gb RAM, 8 cores
Built Apr  4 2023 10:46:17
(C) Copyright 2004-2021 Robert C. Edgar.
https://drive5.com

Input: 82 seqs, avg length 343, max 344

00:00 11Mb   CPU has 8 cores, running 8 threads
00:05 232Mb   100.0% Calc posteriors
00:05 256Mb   100.0% Consistency (1/2)
00:05 257Mb   100.0% Consistency (2/2)
00:05 257Mb   100.0% UPGMA5           
00:05 262Mb   100.0% Refining

muscle 5.2.osxarm64 [6c6011]  25.8Gb RAM, 8 cores
Built Apr  4 2023 10:46:17
(C) Copyright 2004-2021 Robert C. Edgar.
https://drive5.com

Input: 183 seqs, avg length 323, max 327

00:00 12Mb   CPU has 8 cores, running 8 threads


00:22 879Mb   100.0% Calc posteriors
00:23 1.0Gb   100.0% Consistency (1/2)
00:24 918Mb   100.0% Consistency (2/2)
00:24 918Mb   100.0% UPGMA5           
00:25 929Mb   100.0% Refining

muscle 5.2.osxarm64 [6c6011]  25.8Gb RAM, 8 cores
Built Apr  4 2023 10:46:17
(C) Copyright 2004-2021 Robert C. Edgar.
https://drive5.com

Input: 141 seqs, avg length 562, max 568

00:00 11Mb   CPU has 8 cores, running 8 threads


00:38 838Mb   100.0% Calc posteriors
00:39 803Mb   100.0% Consistency (1/2)
00:40 644Mb   100.0% Consistency (2/2)
00:40 644Mb   100.0% UPGMA5           
00:41 660Mb   100.0% Refining


# Assemble Upstream Region Alleleome

Now we have everything we need to construct the alleleome at base-pair resolution

## Load Aligned and Full BLAST Results

Deal with some potential issues here - MG seq is not present, or too many gaps in any of the alignments

In [43]:
MAX_FRAC_GAPS = 0.2
MIN_FRAC_GENOMES = 0.75
MIN_NUM_GENOMES = MIN_FRAC_GENOMES * 2350

five_utr_dir = Path(BLAST_BY_UPSTREAM_REGION_DIR, '5utr')

feat_id_to_align_df = {}
feat_id_to_full_result_df = {}

weird = []
for i, five_utr_align_file in enumerate(os.listdir(five_utr_dir)):
    
    if 'align' not in five_utr_align_file:
        continue
    
    print(f"{i+1}/{len(os.listdir(five_utr_dir))}", end='\r')
    
    feat_id = five_utr_align_file.split('.')[0].split('_align')[0]

    align_result_df = pd.read_csv(Path(five_utr_dir, five_utr_align_file), index_col=0)
    full_result_df = pd.read_csv(Path(five_utr_dir, five_utr_align_file.replace('_align', '')), index_col=0)
    
    # FILTER out sequences that seem to be alignment errors
    align_len = len(align_result_df['seq_align'].iloc[0])
    MAX_FRAC_GAPS = 0.2
    align_result_df = align_result_df[
        align_result_df['seq_align'].apply(lambda s: s.count('-') < align_len * MAX_FRAC_GAPS)
    ]
    
    # want to take those that map well across the pangenome
    if align_result_df['count'].sum() < MIN_NUM_GENOMES:
        continue
    
    mg_feat_seq = mg1655_feat_id_to_seqs[feat_id]
    mg_align_seq = dict(zip(align_result_df['seq'], align_result_df['seq_align'])).get(mg_feat_seq, None)
    
    # don't actually have the MG seq in the listing for some reason
    if mg_align_seq is None:
        weird.append(feat_id)
        continue
    
    # QC step - sometimes wwe somehow get inconsistent amounts of gaps
    if len(mg_align_seq) - mg_align_seq.count('-') != len(mg_feat_seq):
        weird.append(feat_id)
        continue

    feat_id_to_align_df[feat_id] = align_result_df
    feat_id_to_full_result_df[feat_id] = full_result_df

## Determine Widest 5' UTRs

NOTE that in some cases, we can have TSSes that transcribe the same FIRST gene that are not alternates because
they don't transcribe all of the same genes - we just want to get the farthest away of the ones that do indeed
transcribe that first gene

In [44]:
tss_to_align_len = {}

for feat_id, align_df in feat_id_to_align_df.items():
    tss_id = feat_id.split('__')[0]
    tss_to_align_len[tss_id] = len(align_df['seq_align'].iloc[0])

# add in a column we don't have yet - FIRST gene
def get_first_gene(row):
    gene_ids = row['gene_ids']
    tss_gene_df = mg1655.gene_table.loc[row['gene_ids'].split(';')]
    strand = row['strand']
    def dist_to_tss(gene_row):
        if strand == 1:
            return gene_row['left'] - row['tss']
        else:
            return row['tss'] - gene_row['right']
    tss_gene_df = tss_gene_df.assign(
        tss_dist=tss_gene_df.apply(dist_to_tss, axis=1)
    )
    tss_gene_df = tss_gene_df.sort_values(by='tss_dist')
    closest_gene_row = tss_gene_df.iloc[0]
    closest_gene_id = closest_gene_row.name
    return closest_gene_id
    
with_genes = bitome_mg1655_tss_df[bitome_mg1655_tss_df['gene_ids'].notna()]
with_genes = with_genes.assign(
    first_gene_id=with_genes.apply(get_first_gene, axis=1)
)

longest_tsses = []
for first_gene, first_gene_tss_df in with_genes.groupby('first_gene_id'):
    # add in the align length we got before
    first_gene_tss_df = first_gene_tss_df.assign(
        align_len=pd.Series(first_gene_tss_df.index).apply(lambda tss_id: tss_to_align_len.get(tss_id, 0)).values
    )
    longest_id = first_gene_tss_df.sort_values(by='align_len', ascending=False).iloc[0].name
    longest_tsses.append(longest_id)

feat_ids_to_use = []
for feat_id in feat_id_to_align_df.keys():
    tss_id = feat_id.split('__')[0]
    if tss_id in longest_tsses:
        feat_ids_to_use.append(feat_id)

### Compress Widest Region Alignments to File

You'll want to compress the upstream_alignments folder (not done here) as well to prepare it for loading in notebook `1__analyze_alleleome`

In [160]:
wide_region_dir = Path(BLAST_BY_UPSTREAM_REGION_DIR, 'upstream_alignments')

for fid in feat_ids_to_use:
    fid_align_df = feat_id_to_align_df[fid]
    seq_to_allele_id = dict(zip(fid_align_df['seq'], fid_align_df.index))
    fid_full_df = feat_id_to_full_result_df[fid]
    fid_full_df = fid_full_df.assign(
        allele_id=fid_full_df['seq'].apply(lambda s: seq_to_allele_id.get(s, None))
    )
    fid_full_df = fid_full_df.drop(columns=['bitome_feat_id', 'seq'])
    fid_align_df.to_csv(Path(wide_region_dir, f'{fid}_align.csv'))
    fid_full_df.to_csv(Path(wide_region_dir, f'{fid}.csv'))

## Define Alleles

Can use this for phylogenetic analysis

In [45]:
allele_idx = []
allele_pa_rows = []

for i, fid_to_use in enumerate(feat_ids_to_use):
    
    print(f'{i+1}/{len(feat_ids_to_use)}', end='\r')
    
    align_df = feat_id_to_align_df[fid_to_use]
    full_seq_df = feat_id_to_full_result_df[fid_to_use]
    full_seq_qc_df = full_seq_df[full_seq_df['seq'].isin(align_df['seq'])]
    full_seq_qc_df = full_seq_qc_df.assign(
        genome_id=full_seq_qc_df['genome_id'].str.replace('delete', '')
    )
    
    # assumes that the align DF is already ordered by count - we will number the alleles this way
    seq_to_allele_idx_lookup = dict(zip(align_df['seq'], align_df.index))
    
    # map each full seq to its allele ID
    full_seq_qc_df = full_seq_qc_df.assign(
        allele_id=full_seq_qc_df['seq'].apply(lambda s: f'{fid_to_use}__{seq_to_allele_idx_lookup[s]}')
    )
    
    allele_id_to_genome_ids = {aid: {gid: 1 for gid in list(aid_df['genome_id'])} for aid, aid_df in full_seq_qc_df.groupby('allele_id')}
    allele_idx.extend(allele_id_to_genome_ids.keys())
    allele_pa_rows.extend(allele_id_to_genome_ids.values())
    
allele_pa_df = pd.DataFrame(allele_pa_rows, index=allele_idx)

genome_id_to_alleles = {}
for gid in allele_pa_df.columns:
    genome_id_to_alleles[gid] = set(allele_pa_df[gid][allele_pa_df[gid] == 1].index)

In [146]:
with gzip.open(Path(DATA_DIR, 'genome_id_to_alleles.pickle.gz'), 'wb') as f:
    pickle.dump(genome_id_to_alleles, f)

## Assemble Base-Pair Level Variants

In [104]:
# define standard range up and down of TSS as core recognition element
# CRE is -4 to +2 (with the weird counting with no zero)
CRE_UP, CRE_DOWN = 4, 1

# want to record information both as a feature-level summary and at base-pair level
base_pair_rows = []
five_utr_rows = []
for i, feat_id in enumerate(feat_ids_to_use):

    print(f'{i+1}/{len(feat_ids_to_use)}', end='\r')
    tss_id, gene_id, _ = feat_id.split('__')
    tss_row = bitome_mg1655_tss_df.loc[tss_id]
    gene_row = mg1655.gene_table.loc[gene_id]

    if tss_row['strand'] == 1:
        utr_len = gene_row['left'] - tss_row['tss']
    else:
        utr_len = tss_row['tss'] - gene_row['right']
    up_from_gene = FIVE_UTR_TSS_UP + utr_len
    if tss_row['strand'] == 1:
        mg_coord_l, mg_coord_r = gene_row['left'] - up_from_gene, gene_row['left'] + FIVE_UTR_GENE_DOWN
        mg_rel_idx_to_mg_abs_coord = dict(zip(range(-up_from_gene, FIVE_UTR_GENE_DOWN + 1), range(mg_coord_l, mg_coord_r + 1)))
    else:
        mg_coord_l, mg_coord_r = gene_row['right'] - FIVE_UTR_GENE_DOWN, gene_row['right'] + up_from_gene
        mg_rel_idx_to_mg_abs_coord = dict(zip(range(-up_from_gene, FIVE_UTR_GENE_DOWN + 1), range(mg_coord_r, mg_coord_l - 1, -1)))

    align_df = feat_id_to_align_df[feat_id]
    full_result_df = feat_id_to_full_result_df[feat_id]
    full_result_df = full_result_df.assign(
        genome_id=full_result_df['genome_id'].str.replace('delete', '')
    )

    mg_feat_seq = mg1655_feat_id_to_seqs[feat_id]
    mg_align_seq = dict(zip(align_df['seq'], align_df['seq_align'])).get(mg_feat_seq, None)
    align_len = len(mg_align_seq)

    total_seqs = align_df['count'].sum()

    align_pos_to_var_counts = {}
    for align_i, align_bases in enumerate(zip(*align_df['seq_align'])):
        base_count_dict = {}
        for base, count in zip(align_bases, align_df['count']):
            if base in base_count_dict:
                base_count_dict[base] += count
            else:
                base_count_dict[base] = count
        align_pos_to_var_counts[align_i] = dict(
            sorted(base_count_dict.items(), reverse=True, key=lambda tup: tup[1])
        )
        
    # need to be able to interconvert between MG relative pos and align index
    align_idx_to_mg_rel_idx = {}
    current_mg_rel_idx = FIVE_UTR_GENE_DOWN
    current_align_idx = align_len - 1
    while current_align_idx >= 0:
        while mg_align_seq[current_align_idx] == '-':
            align_idx_to_mg_rel_idx[current_align_idx] = current_mg_rel_idx
            current_align_idx -= 1
        align_idx_to_mg_rel_idx[current_align_idx] = current_mg_rel_idx
        current_align_idx -= 1
        if current_mg_rel_idx > -up_from_gene:
            current_mg_rel_idx -= 1
    mg_rel_idx_to_align_idxes = {}
    for align_idx, mg_rel_idx in align_idx_to_mg_rel_idx.items():
        if mg_rel_idx in mg_rel_idx_to_align_idxes:
            mg_rel_idx_to_align_idxes[mg_rel_idx].append(align_idx)
        else:
            mg_rel_idx_to_align_idxes[mg_rel_idx] = [align_idx]

    # --- handle genes in range
    gene_poses_mg_rel = {}
    gene_poses_mg_rel = {rel_pos: gene_row['name'] for rel_pos in range(FIVE_UTR_GENE_DOWN + 1)}
    genes_in_range = mg1655.gene_table[
        # left edge is in here
        ((mg1655.gene_table['left'] >= mg_coord_l) & (mg1655.gene_table['left'] <= mg_coord_r)) | 
        # right edge is in here; this OR covers case where gene is fully within
        ((mg1655.gene_table['right'] >= mg_coord_l) & (mg1655.gene_table['right'] <= mg_coord_r)) | 
        # or gene could totally subsume this range (presumably on oppo strand)
        ((mg1655.gene_table['left'] < mg_coord_l) & (mg1655.gene_table['right'] > mg_coord_r))
    ]
    other_genes_in_range = genes_in_range.drop(index=gene_row.name)
    for _, other_gene_row in other_genes_in_range.iterrows():
        if tss_row['strand'] == 1:
            gene_up_rel = other_gene_row['left'] - gene_row['left']
            gene_down_rel = other_gene_row['right'] - gene_row['left']
        else:
            gene_up_rel = gene_row['right'] - other_gene_row['right']
            gene_down_rel = gene_row['right'] - other_gene_row['left']
        gene_up_cap = max(gene_up_rel, -up_from_gene)
        gene_down_cap = min(gene_down_rel, FIVE_UTR_GENE_DOWN)
        gene_w = gene_down_cap - gene_up_cap + 1
        for gene_rel_pos in range(int(gene_up_cap), int(gene_down_cap + 1)):
            gene_poses_mg_rel[gene_rel_pos] = other_gene_row['name']
        
    # --- Shine-Dalgarno ---
    # small so only check one side
    sd_in_range = mg1655.rbs_table[
        (mg1655.rbs_table['left'] >= mg_coord_l) & (mg1655.rbs_table['left'] <= mg_coord_r)
    ]
    sd_poses_mg_rel = {}
    for _, sd_row in sd_in_range.iterrows():
        if tss_row['strand'] == 1:
            sd_up_rel = sd_row['left'] - gene_row['left']
        else:
            sd_up_rel = gene_row['right'] - sd_row['right']
        sd_w = sd_row['right'] - sd_row['left'] + 1
        for sd_rel_pos in range(int(sd_up_rel), int(sd_up_rel + sd_w)):
            sd_poses_mg_rel[sd_rel_pos] = sd_row.name

    # --- TSS and boxes --- 
    tss_in_range = bitome_mg1655_tss_df[
        (bitome_mg1655_tss_df['tss'] >= mg_coord_l) & (bitome_mg1655_tss_df['tss'] <= mg_coord_r)
    ]
    all_tsses = list(set([tss_id] + list(tss_in_range.index)))
    tss_poses_mg_rel = {}
    cre_poses_mg_rel = {}
    b10_poses_mg_rel = {}
    b35_poses_mg_rel = {}
    spacer_poses_mg_rel = {}
    for this_tss_id in all_tsses:
        this_tss_row = bitome_mg1655_tss_df.loc[this_tss_id]
        tss_loc = this_tss_row['tss']
        if this_tss_row['strand'] == 1:
            tss_rel_idx = tss_loc - gene_row['left']
        else:
            tss_rel_idx = gene_row['right'] - tss_loc
        if tss_rel_idx in tss_poses_mg_rel:
            tss_poses_mg_rel[tss_rel_idx].append(this_tss_id)
        else:
            tss_poses_mg_rel[tss_rel_idx] = [this_tss_id]
        for cre_rel_pos in range(int(tss_rel_idx - CRE_UP), int(tss_rel_idx + CRE_DOWN + 1)):
            if cre_rel_pos in cre_poses_mg_rel:
                cre_poses_mg_rel[cre_rel_pos].append(this_tss_id)
            else:
                cre_poses_mg_rel[cre_rel_pos] = [this_tss_id]

        if pd.notna(this_tss_row['box_10_left']):
            b10_l, b10_r = this_tss_row[['box_10_left', 'box_10_right']]
            if this_tss_row['strand'] == 1:
                b10_up_rel_to_tss_idx = b10_l - tss_loc
            else:
                b10_up_rel_to_tss_idx = tss_loc - b10_r
            b10_up_rel_idx = tss_rel_idx + b10_up_rel_to_tss_idx
            for b10_rel_pos in range(int(b10_up_rel_idx), int(b10_up_rel_idx + (b10_r - b10_l + 1))):
                if b10_rel_pos in b10_poses_mg_rel:
                    b10_poses_mg_rel[b10_rel_pos].append(this_tss_id)
                else:
                    b10_poses_mg_rel[b10_rel_pos] = [this_tss_id]

        if pd.notna(this_tss_row['box_35_left']):
            b35_l, b35_r = this_tss_row[['box_35_left', 'box_35_right']]
            if this_tss_row['strand'] == 1:
                b35_up_rel_to_tss_idx = b35_l - tss_loc
            else:
                b35_up_rel_to_tss_idx = tss_loc - b35_r
            b35_up_rel_idx = tss_rel_idx + b35_up_rel_to_tss_idx
            for b35_rel_pos in range(int(b35_up_rel_idx), int(b35_up_rel_idx + (b35_r - b35_l + 1))):
                if b35_rel_pos in b35_poses_mg_rel:
                    b35_poses_mg_rel[b35_rel_pos].append(this_tss_id)
                else:
                    b35_poses_mg_rel[b35_rel_pos] = [this_tss_id]
                
        # if both are not na, we know we have the right variables assigned
        if pd.notna(this_tss_row['box_35_left']) and pd.notna(this_tss_row['box_10_left']):
            spacer_rel_up = b35_up_rel_idx + (b35_r - b35_l + 1)
            spacer_rel_down = b10_up_rel_idx - 1
            for spacer_rel_pos in range(int(spacer_rel_up), int(spacer_rel_down + 1)):
                if spacer_rel_pos in spacer_poses_mg_rel:
                    spacer_poses_mg_rel[spacer_rel_pos].append(this_tss_id)
                else:
                    spacer_poses_mg_rel[spacer_rel_pos] = [this_tss_id]

    tss_poses_mg_rel = {rel_pos:';'.join(tsses) for rel_pos, tsses in tss_poses_mg_rel.items()}
    cre_poses_mg_rel = {rel_pos:';'.join(tsses) for rel_pos, tsses in cre_poses_mg_rel.items()}
    b10_poses_mg_rel = {rel_pos:';'.join(tsses) for rel_pos, tsses in b10_poses_mg_rel.items()}
    b35_poses_mg_rel = {rel_pos:';'.join(tsses) for rel_pos, tsses in b35_poses_mg_rel.items()}
    spacer_poses_mg_rel = {rel_pos:';'.join(tsses) for rel_pos, tsses in spacer_poses_mg_rel.items()}

    # get the regulatory interactions
    prom_ids = bitome_mg1655_tss_df.loc[all_tsses]['promoter_id']

    ri_df_for_utr = ri_df[ri_df['PROMOTER_ID'].isin(prom_ids)].drop_duplicates(subset='SITE_ID')
    ri_df_for_utr = ri_df_for_utr[ri_df_for_utr['SITE_POSLEFT'].notna()]

    tfbs_poses_mg_rel = {}
    for tf, tf_ri_df in ri_df_for_utr.groupby('T_FACTOR_NAME'):
        for _, tf_site_row in tf_ri_df.iterrows():
            tfbs_l, tfbs_r = tf_site_row[['SITE_POSLEFT', 'SITE_POSRIGHT']]
            if gene_row['strand'] == 1:
                tfbs_rel_up = tfbs_l - gene_row['left']
            else:
                tfbs_rel_up = gene_row['right'] - tfbs_r
            for tfbs_rel_pos in range(int(tfbs_rel_up), int(tfbs_rel_up + tf_site_row['SITE_LENGTH'] + 1)):
                site_tf_id = f"{tf_site_row['SITE_ID']}__{tf}"
                if tfbs_rel_pos in tfbs_poses_mg_rel:
                    tfbs_poses_mg_rel[tfbs_rel_pos].append(site_tf_id)
                else:
                    tfbs_poses_mg_rel[tfbs_rel_pos] = [site_tf_id]
    tfbs_poses_mg_rel = {rel_pos:';'.join(tfs) for rel_pos, tfs in tfbs_poses_mg_rel.items()}
            
    # -- annotate any ALE mutations --- 
    ale_muts_in_range = muts_df[
        (muts_df['Position'] <= mg_coord_r) & (muts_df['Position'] >= mg_coord_l)
    ]
    ale_poses_mg_rel = {}
    for _, ale_mut_row in ale_muts_in_range.iterrows():
        if tss_row['strand'] == 1:
            ale_mut_gene_rel = ale_mut_row['Position'] - gene_row['left']
        else:
            ale_mut_gene_rel = gene_row['right'] - ale_mut_row['Position']
        ale_poses_mg_rel[ale_mut_gene_rel] = str(ale_mut_row.name)
    
    # --- get any attenuators in range
    attenuat_in_range = mg1655.attenuator_table[
        (mg1655.attenuator_table['left'] >= mg_coord_l) & (mg1655.attenuator_table['left'] <= mg_coord_r) | 
        (mg1655.attenuator_table['right'] >= mg_coord_l) & (mg1655.attenuator_table['right'] <= mg_coord_r)
    ]
    attenuate_poses_mg_rel = {}
    for _, atten_row in attenuat_in_range.iterrows():
        if tss_row['strand'] == 1:
            att_up_rel = atten_row['left'] - gene_row['left']
        else:
            att_up_rel = gene_row['right'] - atten_row['right']
        att_w = atten_row['right'] - atten_row['left'] + 1
        for att_rel_pos in range(int(att_up_rel), int(att_up_rel + att_w)):
            if att_rel_pos in attenuate_poses_mg_rel:
                attenuate_poses_mg_rel[att_rel_pos].append(atten_row.name)
            else:
                attenuate_poses_mg_rel[att_rel_pos] = [atten_row.name]
    attenuate_poses_mg_rel = {rel_pos:';'.join(atts) for rel_pos, atts in attenuate_poses_mg_rel.items()}
        
    # --- put it all together ---- assign variant and annotation information to each base pair in the alignment
    for align_idx, align_var_count_dict in align_pos_to_var_counts.items():
        mg_rel_pos = align_idx_to_mg_rel_idx[align_idx]
        bp_row_dict = {
            'feat_id': feat_id,
            'tss_id': tss_id,
            'gene_id': gene_id,
            'gene_name': gene_row['name'],
            'strand': tss_row['strand'],
            'align_idx': align_idx,
            'mg_rel_pos': mg_rel_pos,
            'mg_abs_coord': mg_rel_idx_to_mg_abs_coord[mg_rel_pos],
            'mg_base': mg_align_seq[align_idx],
            'num_seqs': total_seqs,
            # assumes sorted previously
            'dominant_base': list(align_var_count_dict.keys())[0],
            'num_alleles': len(align_var_count_dict),
            'in_gene': gene_poses_mg_rel.get(mg_rel_pos, None),
            'in_gene_bool': mg_rel_pos in gene_poses_mg_rel,
            'in_rbs': sd_poses_mg_rel.get(mg_rel_pos, None),
            'in_rbs_bool': mg_rel_pos in sd_poses_mg_rel,
            'in_tss': tss_poses_mg_rel.get(mg_rel_pos, None),
            'in_tss_bool': mg_rel_pos in tss_poses_mg_rel,
            'in_cre': cre_poses_mg_rel.get(mg_rel_pos, None),
            'in_cre_bool': mg_rel_pos in cre_poses_mg_rel,
            'in_b10': b10_poses_mg_rel.get(mg_rel_pos, None),
            'in_b10_bool': mg_rel_pos in b10_poses_mg_rel,
            'in_b35': b35_poses_mg_rel.get(mg_rel_pos, None),
            'in_b35_bool': mg_rel_pos in b35_poses_mg_rel,
            'in_spacer': spacer_poses_mg_rel.get(mg_rel_pos, None),
            'in_spacer_bool': mg_rel_pos in spacer_poses_mg_rel,
            'in_tfbs': tfbs_poses_mg_rel.get(mg_rel_pos, None),
            'in_tfbs_bool': mg_rel_pos in tfbs_poses_mg_rel,
            'in_ale_mut': ale_poses_mg_rel.get(mg_rel_pos, None),
            'in_ale_mut_bool': mg_rel_pos in ale_poses_mg_rel,
            'in_atten': attenuate_poses_mg_rel.get(mg_rel_pos, None),
            'in_atten_bool': mg_rel_pos in attenuate_poses_mg_rel
        }
        bases = ['A', 'C', 'T', 'G', '-']
        align_var_count_all_dict = {base: align_var_count_dict.get(base, 0) for base in bases}
        bp_row_dict = bp_row_dict | align_var_count_all_dict
        align_var_pct_all_dict = {f'{base}_pct': (align_var_count_dict.get(base, 0) / total_seqs) * 100 for base in bases}
        bp_row_dict = bp_row_dict | align_var_pct_all_dict
        dom_pct = max(align_var_pct_all_dict.values())
        bp_row_dict = bp_row_dict | {
            'dominant_pct': dom_pct,
            'variant_pct': 100 - dom_pct
        }
        
        base_pair_rows.append(bp_row_dict)
        
base_pair_df = pd.DataFrame(base_pair_rows)

in_bool_cols = [col for col in base_pair_df.columns if 'bool' in col]

base_pair_df = base_pair_df.assign(
    in_nothing_bool=~base_pair_df[in_bool_cols].any(axis=1)
)

### Add in Correlates

In [105]:
p1k_gene_table = pd.read_csv(Path(DATA_DIR, 'p1k_gene_table_expanded.csv'), index_col=0)

p1k_median_med, p1k_median_std = p1k_gene_table['median_log_tpm'].median(), p1k_gene_table['median_log_tpm'].std()
p1k_median_borders = p1k_median_med - p1k_median_std, p1k_median_med + p1k_median_std

p1k_mad_med, p1k_mad_std = p1k_gene_table['mad_log_tpm'].median(), p1k_gene_table['mad_log_tpm'].std()
p1k_mad_borders = p1k_mad_med - p1k_mad_std, p1k_mad_med + p1k_mad_std

In [106]:
# add in annotations of various additional correlates
essential_genes = set(mg1655.gene_table[mg1655.gene_table['essential'] == True].index)

def has_essential(gids_raw):
    if pd.notna(gids_raw):
        gids = gids_raw.split(';')
        essential = essential_genes.intersection(set(gids))
        return len(essential) > 0
    else:
        return False
tss_with_essential = bitome_mg1655_tss_df[
    bitome_mg1655_tss_df['gene_ids'].apply(has_essential)
].index
wide_tss_essential = [feat_id for feat_id in feat_ids_to_use if feat_id.split('__')[0] in tss_with_essential]

base_pair_df = base_pair_df.assign(
    essential=base_pair_df['feat_id'].apply(lambda fid: fid in wide_tss_essential)
)

tss_with_exp = bitome_mg1655_tss_df[
    bitome_mg1655_tss_df['median_of_log_tpm_medians'].notna()
].index
wide_feat_w_exp = [feat_id for feat_id in feat_ids_to_use if feat_id.split('__')[0] in tss_with_exp]

tss_to_tpm_median_cat = {}
tss_to_tpm_mad_cat = {}

for feat_id in wide_feat_w_exp:

    tss_id = feat_id.split('__')[0]
    tss_row = bitome_mg1655_tss_df.loc[tss_id]
    tss_median_log_tpm = tss_row['median_of_log_tpm_medians']
    if tss_median_log_tpm < p1k_median_borders[0]:
        tss_to_tpm_median_cat[feat_id] = 'Low'
    elif tss_median_log_tpm < p1k_median_borders[1]:
        tss_to_tpm_median_cat[feat_id] = 'Medium'
    else:
        tss_to_tpm_median_cat[feat_id] = 'High'
        
    median_of_mads = np.median(p1k_gene_table.loc[tss_row['genes_with_log_tpm'].split(';')]['mad_log_tpm'])
    if median_of_mads < p1k_mad_borders[0]:
        tss_to_tpm_mad_cat[feat_id] = 'Low'
    elif median_of_mads < p1k_mad_borders[1]:
        tss_to_tpm_mad_cat[feat_id] = 'Medium'
    else:
        tss_to_tpm_mad_cat[feat_id] = 'High'
        
# separate the base pair DF by expression categories
base_pair_df = base_pair_df.assign(
    tpm_median_cat=base_pair_df['feat_id'].apply(lambda fid: tss_to_tpm_median_cat[fid]),
    tpm_mad_cat=base_pair_df['feat_id'].apply(lambda fid: tss_to_tpm_mad_cat[fid])
)

In [107]:
base_pair_df.to_csv(Path(DATA_DIR, 'alleleome_base_pairs.csv.tar.gz'))